In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
%%appyter hide_code

{% do SectionField(
    name='primary',
    title='Upload Tumor Expression',
    img='upload.png'
) %}


{% set rna_file = CustomFileField(
    name='rna_expr',
    label='RNA-seq expression',
    description='''
    File should be a tsv/csv of the form:

        Patient 1 Tumor    Patient 2 Tumor  ...
    ---------------------------------------
    Gene 1    0                      200                ...
    ---------------------------------------
    Gene 2    5                      180                ...
    ---------------------------------------
      ...           ...                      ...                  ...
    ''',
    default='LUAD_tumor_counts.tsv',
    required=False,
    examples={
        'BR_tumor_counts.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/BR_tumor_counts.tsv',
        'CCRCC_tumor_counts.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/CCRCC_tumor_counts.tsv',
        'CO_tumor_counts.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/CO_tumor_counts.tsv',
        'GBM_tumor_counts.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/GBM_tumor_counts.tsv',
        'HNSCC_tumor_counts.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/HNSCC_tumor_counts.tsv',
        'LSCC_tumor_counts.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/LSCC_tumor_counts.tsv',
        'LUAD_tumor_counts.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/LUAD_tumor_counts.tsv',
        'OV_tumor_counts.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/OV_tumor_counts.tsv',
        'PDAC_tumor_counts.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/PDAC_tumor_counts.tsv',
        'UCEC_tumor_counts.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/UCEC_tumor_counts.tsv'
    },
    section='primary',
) %}


{% set prot_file = CustomFileField(
    name='prot_expr',
    label='protein expression',
    description='''
    File should be a tsv/csv of the form:

        Patient 1 Tumor    Patient 2 Tumor  ...
    ---------------------------------------
    Gene/Protein 1    0                   200       ...
    ---------------------------------------
    Gene/Protein 2    5                   180       ...
    ---------------------------------------
      ...                       ...                    ...        ...
    ''',
    default='LUAD_proteomics_tumor.tsv',
    required=False,
    examples={
        'BR_proteomics_tumor.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/BR_proteomics_tumor.tsv',
        'CCRCC_proteomics_tumor.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/CCRCC_proteomics_tumor.tsv',
        'CO_proteomics_tumor.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/CO_proteomics_tumor.tsv',
        'GBM_proteomics_tumor.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/GBM_proteomics_tumor.tsv',
        'HNSCC_proteomics_tumor.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/HNSCC_proteomics_tumor.tsv',
        'LSCC_proteomics_tumor.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/LSCC_proteomics_tumor.tsv',
        'LUAD_proteomics_tumor.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/LUAD_proteomics_tumor.tsv',
        'OV_proteomics_tumor.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/OV_proteomics_tumor.tsv',
        'PDAC_proteomics_tumor.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/PDAC_proteomics_tumor.tsv',
        'UCEC_proteomics_tumor.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/UCEC_proteomics_tumor.tsv'
    },
    section='primary',
) %}

{% set phospho_file = CustomFileField(
    name='phospho_expr',
    label='protein phosphorylation',
    description='''
    File should be a tsv/csv of the form:

        Patient 1 Tumor    Patient 2 Tumor  ...
    ---------------------------------------
    Phosphoprotein 1     0        200          ...
    ---------------------------------------
    Phosphoprotein 2    5         180           ...
    ---------------------------------------
      ...                          ...              ...          ...
    ''',
    default='LUAD_phospho_observed.tsv',
    required=False,
    examples={
        'BR_phospho_observed.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/BR_phospho_observed.tsv',
        'CCRCC_phospho_observed.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/CCRCC_phospho_observed.tsv',
        'CO_phospho_observed.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/CO_phospho_observed.tsv',
        'GBM_phospho_observed.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/GBM_phospho_observed.tsv',
        'HNSCC_phospho_observed.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/HNSCC_phospho_observed.tsv',
        'LSCC_phospho_observed.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/LSCC_phospho_observed.tsv',
        'LUAD_phospho_observed.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/LUAD_phospho_observed.tsv',
        'OV_phospho_observed.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/OV_phospho_observed.tsv',
        'PDAC_phospho_observed.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/PDAC_phospho_observed.tsv',
        'UCEC_phospho_observed.tsv': 'https://minio.dev.maayanlab.cloud/x2k-tr/UCEC_phospho_observed.tsv',
        
    },
    section='primary',
) %}

{% set meta_file = CustomFileField(
    name='meta',
    label='sample metadata',
    description='Samples on the rows, metadata on the columns',
    default='LUAD_meta.txt',
    required=True,
    examples={
        'BR_meta.txt': 'https://minio.dev.maayanlab.cloud/x2k-tr/BR_meta.txt',
        'CCRCC_meta.txt': 'https://minio.dev.maayanlab.cloud/x2k-tr/CCRCC_meta.txt',
        'CO_meta.txt': 'https://minio.dev.maayanlab.cloud/x2k-tr/CO_meta.txt',
        'GBM_meta.txt': 'https://minio.dev.maayanlab.cloud/x2k-tr/GBM_meta.txt',
        'HNSCC_meta.txt': 'https://minio.dev.maayanlab.cloud/x2k-tr/HNSCC_meta.txt',
        'LSCC_meta.txt': 'https://minio.dev.maayanlab.cloud/x2k-tr/LSCC_meta.txt',
        'LUAD_meta.txt': 'https://minio.dev.maayanlab.cloud/x2k-tr/LUAD_meta.txt',
        'OV_meta.txt': 'https://minio.dev.maayanlab.cloud/x2k-tr/OV_meta.txt',
        'PDAC_meta.txt': 'https://minio.dev.maayanlab.cloud/x2k-tr/PDAC_meta.txt',
        'UCEC_meta.txt': 'https://minio.dev.maayanlab.cloud/x2k-tr/UCEC_meta.txt'
    },
    section='primary',
) %}

{% set membrane_screener_ = BoolField(
    name='membrane_screener',
    label='Prioritize membrane genes',
    description='Use a list of genes that encode cell surface proteins to identify targets that might be suitable for immunotherapy.',
    default=True,
    section='primary',
) %}

{% set normalize_to_bg = BoolField(
    name='normalize_to_bg',
    label='Normalize to background',
    description='Normalize the uploaded RNA-seq expression values to the background expression values for target identification through differential gene expression analysis',
    default=True,
    section='primary',
) %}

{% set impute_protein_expr = BoolField(
    name='impute_protein_expr',
    label='Impute protein expression',
    description='Impute missing protein expression values with average of non-missing values in rows with at least 80% non-missing values',
    default=True,
    section='primary',
) %}

{% set impute_phospho_expr = BoolField(
    name='impute_phospho_expr',
    label='Impute protein phosphorylation',
    description='Impute missing protein phosphorylation values with average of non-missing values in rows with at least 80% non-missing values',
    default=True,
    section='primary',
) %}

{% set percent_rows_protein = FloatField(
    name='percent_rows_protein',
    label='Protein imputation threshold', 
    default=0.8, 
    min=0.0,
    max=1,
    step=0.01,
    description='Rows without at least this ratio of non-null values will be dropped for imputation', 
    section='primary'
)
%}

{% set percent_rows_phospho = FloatField(
    name='percent_rows_phospho',
    label='Phosphoprotein imputation threshold', 
    default=0.5, 
    min=0.0,
    max=1,
    step=0.01,
    description='Rows without at least this ratio of non-null values will be dropped for imputation', 
    section='primary'
)
%}


{% set chea3_n = IntField(
    name='chea3_n',
    label='DEGs for ChEA3 Enrichment', 
    default=500, 
    min=10,
    max=2000,
    step=1,
    description='Number of genes per patient to be submitted to ChEA3 for enrichment analysis as part of the X2K pipeline.', 
    section='primary'
)
%}

{% set kea3_n = IntField(
    name='kea3_n',
    label='DEPPs for KEA3 Enrichment', 
    default=500, 
    min=10,
    max=2000,
    step=1,
    description='Number of differentially phosphorylated proteins per patient to be submitted to KEA3 for enrichment analysis.', 
    section='primary'
)
%}

{% set n_neighbors = IntField(
    name='n_neighbors',
    label='N neighbors', 
    default=15, 
    min=2,
    max=100,
    step=1,
    description='The size of local neighborhood (in terms of number of neighboring data points) used for manifold approximation.', 
    section='primary'
)
%}

{% set min_dist = FloatField(
    name='min_dist',
    label='Min Distance', 
    default=0.01, 
    min=0.00001,
    max=0.9,
    step=0.00001,
    description='The effective minimum distance between embedded points.', 
    section='primary'
)
%}

{% set resolution = FloatField(
    name='resolution',
    label='Resolution', 
    default=1, 
    min=0.1,
    max=2,
    step=0.1,
    description='A parameter value controlling the coarseness of the clustering. Higher values lead to more clusters.', 
    section='primary'
)
%}

{% set tumor_validate = MultiChoiceField(
    name='tumor_validate',
    label='DepMap cell lines for target validation', 
    default=[], 
    description='If using a cancer dataset, choose the tumor type(s) in which you would like to validate the predicted kinases and gene targets. The appropriate DepMap cell lines will be used to quantify the effect of gene knock-down on cancer cell viability.', 
    choices=['Acute Myeloid Leukemia', 'Adenosquamous Carcinoma of the Pancreas', 'Ampullary Carcinoma', 'Anaplastic Thyroid Cancer', 'B-Lymphoblastic Leukemia/Lymphoma', 'Bladder Squamous Cell Carcinoma', 'Bladder Urothelial Carcinoma', 'Breast Ductal Carcinoma In Situ', 'Breast Neoplasm, NOS', 'Cervical Adenocarcinoma', 'Cervical Squamous Cell Carcinoma', 'Chondrosarcoma', 'Colorectal Adenocarcinoma', 'Cutaneous Squamous Cell Carcinoma', 'Diffuse Glioma', 'Embryonal Tumor', 'Endometrial Carcinoma', 'Epithelioid Sarcoma', 'Esophageal Squamous Cell Carcinoma', 'Esophagogastric Adenocarcinoma', 'Ewing Sarcoma', 'Extra Gonadal Germ Cell Tumor', 'Fibrosarcoma', 'Gestational Trophoblastic Disease', 'Glassy Cell Carcinoma of the Cervix', 'Head and Neck Carcinoma, Other', 'Head and Neck Squamous Cell Carcinoma', 'Hepatoblastoma', 'Hepatocellular Carcinoma', 'Hepatocellular Carcinoma plus Intrahepatic Cholangiocarcinoma', 'Hodgkin Lymphoma', 'Intracholecystic Papillary Neoplasm', 'Intraductal Papillary Neoplasm of the Bile Duct', 'Invasive Breast Carcinoma', 'Leiomyosarcoma', 'Liposarcoma', 'Lung Neuroendocrine Tumor', 'Mature B-Cell Neoplasms', 'Mature T and NK Neoplasms', 'Melanoma', 'Meningothelial Tumor', 'Merkel Cell Carcinoma', 'Mixed Cervical Carcinoma', 'Mucosal Melanoma of the Vulva/Vagina', 'Myelodysplastic Syndromes', 'Myeloproliferative Neoplasms', 'Nerve Sheath Tumor', 'Neuroblastoma', 'Non-Cancerous', 'Non-Hodgkin Lymphoma', 'Non-Seminomatous Germ Cell Tumor', 'Non-Small Cell Lung Cancer', 'Ocular Melanoma', 'Osteosarcoma', 'Ovarian Epithelial Tumor', 'Ovarian Germ Cell Tumor', 'Pancreatic Adenocarcinoma', 'Pleural Mesothelioma', 'Poorly Differentiated Thyroid Cancer', 'Prostate Adenocarcinoma', 'Renal Cell Carcinoma', 'Retinoblastoma', 'Rhabdoid Cancer', 'Rhabdomyosarcoma', 'Salivary Carcinoma', 'Sarcoma, NOS', 'Squamous Cell Carcinoma of the Vulva/Vagina', 'Synovial Sarcoma', 'T-Lymphoblastic Leukemia/Lymphoma', 'Undifferentiated Pleomorphic Sarcoma/Malignant Fibrous Histiocytoma/High-Grade Spindle Cell Sarcoma', 'Urethral Cancer', 'Uterine Sarcoma/Mesenchymal', 'Well-Differentiated Thyroid Cancer'],
    section='primary'
)
%}

{% set membrane_screener_list = 'https://appyters.maayanlab.cloud/storage/Tumor_Gene_Target_Screener/surfaceome.csv' %}


In [ ]:
from helpers import *
import os
import re
import random
import qnorm
from string import ascii_uppercase
from tqdm import tqdm
import numpy as np
import pandas as pd
import scanpy as sc

from matplotlib import cm
from matplotlib.patches import Rectangle
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
%matplotlib inline

import networkx as nx

from IPython.display import HTML, display, Markdown, FileLink
from matplotlib.gridspec import GridSpec
from matplotlib_venn import venn2
from maayanlab_bioinformatics.normalization import zscore_normalize, log2_normalize
from scipy.stats import zscore
from scipy.stats import ttest_ind, ttest_1samp, ttest_ind_from_stats
from maayanlab_bioinformatics.dge import limma_voom_differential_expression
from maayanlab_bioinformatics.harmonization.ncbi_genes import ncbi_genes_lookup
from maayanlab_bioinformatics.api import enrichr_link_from_genes
from statsmodels.stats.multitest import multipletests

import warnings
warnings.filterwarnings("ignore")

os.makedirs('results', exist_ok=True)
os.makedirs('figures', exist_ok=True)
try:
    lookup = ncbi_genes_lookup(organism='Mammalia/Homo_sapiens')
except:
    import urllib.request, json 
    with urllib.request.urlopen("https://minio.dev.maayanlab.cloud/x2k-tr/ncbi_genes_disambiguated.json") as url:
        ncbi_genes_disambiguated = json.load(url)
        lookup = ncbi_genes_disambiguated.get
    
fig_counter = 2
table_counter = 1
discussion_results = {}

import sys
import contextlib
@contextlib.contextmanager
def suppress_output(stdout=True, stderr=True, dest=os.devnull):
    ''' Usage:
    with suppress_output():
        print('hi')
    '''
    dev_null = open(dest, 'a')
    if stdout:
        _stdout = sys.stdout
        sys.stdout = dev_null
    if stderr:
        _stderr = sys.stderr
        sys.stderr = dev_null
    try:
        yield
    finally:
        if stdout:
            sys.stdout = _stdout
        if stderr:
            sys.stderr = _stderr

In [ ]:
%%appyter code_exec
has_rna_file = {{ rna_file }} != ''
has_prot_file = {{ prot_file }} != ''
has_phospho_file = {{ phospho_file }} != ''
has_meta_file = {{ meta_file }} != ''

In [ ]:
%%appyter code_exec
if has_rna_file:
    rna_df = read_table({{ rna_file }})
    rna_filename = {{ rna_file }}.split('.')[0]
if has_prot_file:
    prot_df = read_table({{ prot_file }})
    prot_filename = {{ prot_file }}.split('.')[0]
if has_phospho_file:
    phospho_df = read_table({{ phospho_file }})
    phospho_filename = {{ phospho_file }}.split('.')[0]
if has_meta_file:
    meta_df = read_table({{ meta_file }})
else:
    meta_df = pd.DataFrame()

In [ ]:
example_mapper = {
    'BR': 'breast invasive carcinoma (BR)', 
    'CCRCC': 'clear cell renal cell carcinoma (ccRCC)',
    'CO': '	colon adenocarcinoma (COAD)',
    'GBM': 'glioblastoma multiforme (GBM)',
    'HNSCC': 'head and neck squamous cell carcinoma (HNSCC)',
    'LSCC': 'laryngeal squamous cell carcinoma (LSCC)',
    'LUAD': 'lung adenocarcinoma (LUAD)',
    'OV': 'ovarian serous cystadenocarcinoma (OV)',
    'PDAC': 'pancreatic adenocarcinoma (PDAC)',
    'UCEC': 'uterine corpus endometrial carcinoma (UCEC)'
 }

In [ ]:
if has_rna_file and has_phospho_file and has_phospho_file:
    example_name = rna_filename.split('_')[0] + ' Samples' if rna_filename.split('_')[0] in example_mapper else ' Cancer'
    display(Markdown(f'## Multiomics2Targets: Computational Workflow to Identify Pathways and Targets for {example_name} Profiled with Transcriptomics, Proteomics, and Phosphoproteomics'))
elif has_rna_file and has_phospho_file:
    example_name = rna_filename.split('_')[0] + ' Samples' if rna_filename.split('_')[0] in example_mapper else ' Cancer'
    display(Markdown(f'## Multiomics2Targets: Computational Workflow to Identify Pathways and Targets for {example_name} Profiled with Transcriptomics and Phosphoproteomics'))
elif has_rna_file and has_prot_file:
    example_name = rna_filename.split('_')[0] + ' Samples' if rna_filename.split('_')[0] in example_mapper else ' Cancer'
    display(Markdown(f'## Multiomics2Targets: Computational Workflow to Identify Pathways and Targets for {example_name} Profiled with Transcriptomics and Proteomics'))
elif has_phospho_file and has_prot_file:
    example_name = phospho_filename.split('_')[0] + ' Samples' if phospho_filename.split('_')[0] in example_mapper else ' Cancer'
    display(Markdown(f'## Multiomics2Targets: Computational Workflow to Identify Pathways and Targets for {example_name} Profiled with Phosphoproteomics and Proteomics'))
elif has_rna_file:
    example_name = rna_filename.split('_')[0] + ' Samples' if rna_filename.split('_')[0] in example_mapper else ' Cancer'
    display(Markdown(f'## Multiomics2Targets: Computational Workflow to Identify Pathways and Targets for {example_name} Profiled with Transcriptomics'))
elif has_prot_file:
    example_name = prot_filename.split('_')[0] + ' Samples' if prot_filename.split('_')[0] in example_mapper else ' Cancer'
    display(Markdown(f'## Multiomics2Targets: Computational Workflow to Identify Pathways and Targets for {example_name} Profiled with Proteomics'))
elif has_phospho_file:
    example_name = phospho_filename.split('_')[0] + ' Samples' if phospho_filename.split('_')[0] in example_mapper else ' Cancer'
    display(Markdown(f'## Multiomics2Targets: Computational Workflow to Identify Pathways and Targets for {example_name} Profiled with Phosphoproteomics'))
else:
    display(Markdown(f'## Multiomics2Targets: Insert Subtitle Here'))


In [ ]:
display(Markdown('## Abstract'))
abstract_text = ""
abstract_text += '''A major goal in cancer research is to identify cell signaling pathways and tumor- specific proteins that are targetable with immunotherapies such as antibody drug conjugates (ADCs) and chimeric antigen receptor (CAR) T-cells, as well as kinase inhibitors. Multiomics2Targets is a computational pipeline that identifies proteins, genes, and transcripts that are highly expressed in tumors compared to most healthy human tissues and cell types while prioritizing cell-surface proteins and protein kinases. 
To identify driver cell signaling pathways and potential immunotherapeutic candidates the pipeline applies the eXpression to Kinases (X2K) [1, 2] and TargetRanger [3] algorithms.'''
if has_rna_file:
    abstract_text += f" The {rna_df.shape[1]} {example_mapper.get(rna_filename.split('_')[0], 'tumor')} samples are first clustered into subtypes and then cell signaling pathways are inferred from the RNA-seq data with X2K for each cluster. Then cell surface targets are prioritized for each cluster compared to healthy tissue and cell type expression."
if has_rna_file and has_prot_file:
    abstract_text += ' These targets are subsequently confirmed for high protein expression in the tumor subtypes based on the proteomics profiling of the same tumors.'
if has_phospho_file:
    abstract_text += ' Differentially phosphorylated proteins are identified in the tumor samples and used to infer kinase activity with KEA3.'
if has_phospho_file and has_rna_file:
    abstract_text += ' The inferred kinase activity is then used to identify potential kinase targets, comparing them to the driver kinases identified from the transcriptomics through the X2K pipeline.'
if has_phospho_file and has_prot_file and has_rna_file:
    abstract_text += ' Samples with significant recovery of kinases inferred from the X2K pipeline are compared to the kinases inferred from the phosphoproteomics data are then visualized with the matched proteomics profiling.'
abstract_text += ''' With Multiomics2Targets we can identify a multitude of potential driver pathways and novel targets for each cancer subtype. Some of these targets are supported by prior publications while many others are new and warrant further investigation. 
Overall, the Multiomics2Targets pipeline is a robust computational approach to harness knowledge from multi-omics datasets to identify promising immunotherapeutic and other targets for preclinical cancer research.'''
display(Markdown(abstract_text))

In [ ]:
# Load the buttons

## Introduction

A prominent challenge in cancer immunotherapy and targeted therapy is identifying viable targets and drugs that can selectively kill, or stop the proliferation, of tumor cells without negatively impacting normal cells and tissues. Furthermore, personalized cancer therapy requires that target and drug candidates are tailored to individual or subgroups of tumors. These pathways and targets can be characterized as the molecular alterations of regulatory mechanisms, which are known to vary across different tumor types and individuals [4]. Many computational methods have been developed to elucidate key molecular mechanisms, identify targets and pathways in individual tumors. For example, we developed the eXpression2Kinases (X2K) pipeline [1, 2]. The X2K pipeline takes a set of differentially expressed genes and infers the upstream regulatory network likely responsible for the observed changes in gene expression upstream by chaining the tools ChEA3 [5], Genes2Networks [6], and KEA3 [7]. Other and algorithms for pathway analysis from RNA-seq often use gene expression as a proxy for protein activity. Often such methods are limited to identifying specific curated cell signaling pathways [8, 9]. By contrast, X2K chains several enrichment and network analysis steps to reconstruct an original pathway from information encoded in transcription-factor/target, protein-protein interactions, and kinase-substrate networks. Once such upstream cell signaling pathways have been identified, these can be targeted, for example, by knocking out the top ranked kinases from last step of the X2K workflow [10].


Computational approaches have been developed to identify cell surface protein targets for a specific group of cancer patients profiled with RNA-seq with experimental demonstrations that these approaches can work. For example, to identify immunotherapuetic targets for a neuroblastoma patient [11], gene expression data from the patient’s tumor were compared to RNA-seq data from GTEx [12]. Targets were prioritized if they were highly expressed in the tumor compared with their expression across 54 normal GTEx tissues. Genes that met this criterion were further ﬁltered for those that encode cell-surface proteins because these can be targeted by immunotherapies such as antibody-drug conjugate (ADC) [13] or chimeric antigen receptor (CAR) T-cell [14, 15]. The final candidate, GPC2, was experimentally validated by demonstrating that a GPC2-targeted ADC safely and effectively delivered cytotoxic drugs to eliminate neuroblastoma cells in-vitro and in-vivo. Similarly, another study [16] compared tissue-level gene expression from 12 pediatric cancers to normal tissue samples. Through individual identification of highly expressed membrane proteins in the cancerous tissues, targetable tumor antigens were identified. Novel candidates as well as targets that are currently being tested in clinical trials were identified, including GPC2. Other studies integrated transcriptomics and proteomics profiling to identify and rank differentially expressed cell surface proteins in prostate cancer [17] and multiple myeloma [18] cell lines. To make such approaches accessible to users without computational background, we developed TargetRanger [3], a bioinformatics webserver application that identified highly expressed genes from a tumor sample by comparing the expression levels in the tumor to the expression of the same genes in GTEx [12], ARCHS4 [19], and the Tabula sapiens [20] atlases.

Here, we developed a workflow called Multiomics2Targets by applying X2K and TargetRanger to the transcriptomics, proteomics, and phosphoproteomics data collected from cohorts of cancer patients. The workflow first performs pathway analysis of the tumor samples using the X2K algorithm. The workflow accepts gene expression profiles from a collection of tumors, performs clustering analysis on the expression data matrix, and then identifies cell signaling pathways that are most likely activated and inhibited in each cluster. Since the X2K pipeline produces a ranked list of protein kinases, these ranked kinases are compared to ranked kinases inferred from applying the KEA3 tool [7] directly to the phosphoproteomics data collected from the same cluster of patients. The kinases that are identified by both tools are likely the most relevant to target.
Then cell surface targets that are highly expressed in the tumor compared to the expression of these targets in normal cell types and tissues from different atlases are identified with TargetRanger [3]. The entire Multiomics2Targets workflow is illustrated in the diagram below (Fig.  1).


In [ ]:
%%appyter markdown

{% if rna_file.value != '' and prot_file.value != '' and phospho_file.value != '' %}
<img src='https://minio.dev.maayanlab.cloud/x2k-tr/X2KTR_workflows.png' alt="Multiomics2Targets Workflow Diagram">
{% elif rna_file.value != '' and prot_file.value != '' %}
<img src='https://minio.dev.maayanlab.cloud/x2k-tr/X2KTR_workflows-rna-prot.png' alt="Multiomics2Targets Workflow Diagram">
{% elif rna_file.value != '' and phospho_file.value != '' %}
<img src='https://minio.dev.maayanlab.cloud/x2k-tr/X2KTR_workflows-rna-phospho.png' alt="Multiomics2Targets Workflow Diagram">
{% elif phospho_file.value != '' and prot_file.value != '' %}
<img src='https://minio.dev.maayanlab.cloud/x2k-tr/X2KTR_workflows-prot-phospho.png' alt="Multiomics2Targets Workflow Diagram">
{% elif phospho_file.value != '' %}
<img src='https://minio.dev.maayanlab.cloud/x2k-tr/X2KTR_workflows-phospho.png' alt="Multiomics2Targets Workflow Diagram">
{% elif prot_file.value != '' %}
<img src='https://minio.dev.maayanlab.cloud/x2k-tr/X2KTR_workflows-prot.png' alt="Multiomics2Targets Workflow Diagram">
{% else %}
<img src='https://minio.dev.maayanlab.cloud/x2k-tr/X2KTR_workflows-rna.png' alt="Multiomics2Targets Workflow Diagram">
{% endif %}

In [ ]:
%%appyter code_exec
{% if rna_file.value != '' and prot_file.value != '' and phospho_file.value != '' %}
display(Markdown("__Fig. 1__ Workflow of the Multiomics2Targets pipeline. The pipeline utilizes transcriptomics, phosphoproteomics, and proteomics data matrices from tumor samples to identify driver kinases and potential cell-surface targets."))
{% elif rna_file.value != '' and prot_file.value != '' %}
display(Markdown("__Fig. 1__ Workflow of the Multiomics2Targets pipeline. The pipeline utilizes transcriptomics and proteomics data matrices from tumor samples to identify driver kinases and potential cell-surface targets."))
{% elif rna_file.value != '' and phospho_file.value != '' %}
display(Markdown("__Fig. 1__ Workflow of the Multiomics2Targets pipeline. The pipeline utilizes transcriptomics and phosphoproteomics data matrices from tumor samples to identify driver kinases and potential cell-surface targets."))
{% elif phospho_file.value != '' and prot_file != '' %}
display(Markdown("__Fig. 1__ Workflow of the Multiomics2Targets pipeline. The pipeline utilizes proteomics and phosphoproteomics data matrices from tumor samples to identify driver kinase that could serve as potential targets."))
{% elif phospho_file.value != '' %}
display(Markdown("__Fig. 1__ Workflow of the Multiomics2Targets pipeline. The pipeline utilizes phosphoproteomics data matrices from tumor samples to identify driver kinases that could serve as potential targets."))
{% elif prot_file.value != '' %}
display(Markdown("__Fig. 1__ Workflow of the Multiomics2Targets pipeline. The pipeline utilizes proteomics data matrices from tumor samples to identify highly variable proteins in tumor samples."))
{% else %}
display(Markdown("__Fig. 1__ Workflow of the Multiomics2Targets pipeline. The pipeline utilizes transcriptomics data matrices from tumor samples to identify driver kinases and potential cell-surface targets."))
{% endif %}

In [ ]:
%%appyter code_exec
percent_rows_phospho = {{ percent_rows_phospho }}
percent_rows_protein = {{ percent_rows_protein }}
chea3_n = {{ chea3_n }}
kea3_n = {{ kea3_n }}
n_neighbors = {{ n_neighbors }}
min_dist = {{ min_dist }}
resolution = {{ resolution }}
tumor_validate = {{ tumor_validate }}
default_meta_cols = ['Sex', 'Tumor_Size_cm', 'Stage', 'Tobacco_smoking_history']

## Methods

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
*Clustering Samples from the RNA-seq Matrix*

Grouping patients together who share similar vectors of expression may lead to identification of targets specific to smaller subsets of patients. Clustering is performed on the 5000 most variable log2 normalized genes using the Leiden community algorithm [21] 

*Differentially Expressed Genes*

Differentially expressed genes (DEGs) are computed limma voom [22] for each cluster versus all the other samples. Additionally, differentially expressed genes are submitted to Enrichr [23] for identification of biological functions from Gene Ontology (GO BP) [24], pathways from Wikipathways [25], and phenotypes from the Mouse Genome Informatics (MGI) Mammmalian Phenotype Onotology [26].

*Consensus Cluster Labels*

We utilize GPT-4 and the significantly enriched terms from Enrichr to compute 'consensus' labels for each identified cluster. 
The model is instructed to summarize the significant terms from the up- and down-regulated genes in five words or less. 
The system prompt is specified as follows: "You are an assistant to a biologist who has performed enrichment analysis on a set of up-regulated and down-regulated genes for a certain cluster of patient samples. You must determine a consensus label for a given cluster based on the significantly enriched terms which relate to biological processes and phenotypes." The user prompt on each run is then formatted as follows:
"The most significantly enriched terms for the upregulated genes of cluster {cluster} are: {', '.join(up_enrichments)}. For the down genes the significantly enriched terms are: {', '.join(down_enrichments)}. Please provide a consensus label for this cluster with no other reasoning. The label should be at maximum 5 words in length."

*Membrane Proteins Filter*

A membrane/surface protein filter was developed from the intersection of two data sources:
[COMPARTMENTS](https://compartments.jensenlab.org/About) [27] knowledge predictions for human genes where a filter was applied for "Plasma membrane" and "Cell surface" subcellular localization with a confidence score greater than or equal to 3, and the [Human Protein Atlas](https://www.proteinatlas.org/) [28] membrane proteins where a filter was applied for "Evidence at protein level" and removal of genes with "Low tissue specificity."

*Cell Surface Target Identification*

We utilize the TargetRanger [3] to identify genes that are highly expressed in the target cells while lowly expressed across normal human cell types and tissues. TargetRanger [3] compares the inputted samples with processed RNA-seq from several atlases containing healthy tissue and cell type gene expression, namely, GTEx [12], ARCHS4 [19], and Tabula Sapiens [20].

*Transcription Factor Enrichment Analysis with ChEA3*

To assess enriched transcription factors (TFs) from individual samples, we normalized and Z-scored the uploaded RNA-seq data, and then submitted the top 500 upregulated and downregulated genes for each sample to ChEA3 [5] to retrieve ranked lists of TFs.

*Compute TF intermediates using Protein-Protein Co-expression*

To identify additional proteins that connect transcription factors to upstream kinases we utilize protein-protein co-expression from the Cancer Cell Line Encyclopedia (CCLE) [29] to augment the top-ranked enriched transcription factors.

*Infering Kinases using KEA3 from TFs + Intermediates*

To infer upstream kinases in the (Expression to Kinases) X2K [1,2] pipeline we submitted the co-expressed intermediates and enriched transcription factors to KEA3 [7].
{% endif %}

{% if phospho_file.value != '' %}
*Phosphoproteomics Analysis*

From the phosphoproteomic data, we can identify upregulated and downregulated kinases using kinase enrichment analysis from KEA3 [7], and then compare those kinases to the ones identified by the X2K analysis.
{% endif %}

{% if prot_file.value != '' %}
*Proteomics Analysis*

Using the uploaded proteomics data, we can further validate the kinases and cell-surface protein targets identified in the pipeline to ensure the proteins are also expressed at the protein level after Z-scoring the matrix. 
{% endif %}

{% if rna_file.value != '' and tumor_validate != [] %}
*Validation of Cancer Targets*
To gain insight into target efficacy, the DepMap database [30] provides gene effect scores that quantify the effect of gene knock-down on cell viability. Gene effect scores are averaged from the cancer cell lines that match the specified tumor type.
To gain insight into target novelty, we query PubMed for the number of literature co-occurrences that exist between the gene target and the specified cancer.  
{% endif %}

## Results

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Clustering Samples from the RNA-seq Expression Matrix
{% endif %}

In [ ]:
if has_rna_file:
    rna_df.index = rna_df.index.map(lambda g: g.split('.')[0])
    rna_df.index = rna_df.index.map(lambda g: lookup(g) if lookup(g) else g)

In [ ]:
if has_rna_file:    
    adata = sc.AnnData(log2_normalize(qnorm.quantile_normalize(rna_df)).T.values, dtype="float")
    adata.var['gene_names'] = rna_df.index.values
    adata.obs['samples'] = rna_df.columns.values
    # Sort genes by variance
    adata.var['var_rank'] = (-np.var(adata.X, axis=0, dtype="float")).argsort()
    adata = adata[:, adata.var.var_rank < 5000]

    # UMAP
    sc.pp.pca(adata)
    sc.pp.neighbors(adata, n_neighbors=n_neighbors)  # create neighborhood graph
    sc.tl.umap(adata, min_dist=min_dist, alpha=0.1)  # embed umap based on neighborhood graph
    sc.tl.leiden(adata, resolution=resolution)  # clustering
    clusters = adata.obs['leiden'].unique()
    cmap = plt.cm.get_cmap('tab20b', len(clusters))
    for i, cluster in enumerate(sorted(clusters)):
        mask = adata.obs['leiden'] == cluster
        ax = plt.scatter(x=adata.obsm['X_umap'][mask, 0], y=adata.obsm['X_umap'][mask, 1], 
                    color=cmap(i), label=f'Cluster {cluster}', s=10)
    plt.legend(title='Leiden clusters', bbox_to_anchor=(1.05, 1), loc='best')
    ax.axes.set_xticks([])
    ax.axes.set_yticks([])
    plt.xlabel('UMAP-1')
    plt.ylabel('UMAP-2')
    plt.tight_layout()
    plt.savefig(f'figures/{rna_filename}_umap_leiden.png')
    plt.savefig(f'figures/{rna_filename}_umap_leiden.svg')

In [ ]:
if has_rna_file:    
    display(Markdown(f"__Fig. {fig_counter}__ Leiden clusters determined from log2 RNA-seq expression for 5000 most variable genes."))
    display(FileLink(f'figures/{rna_filename}_umap_leiden.png', result_html_prefix='Leiden clusters UMAP PNG: '))
    display(FileLink(f'figures/{rna_filename}_umap_leiden.svg', result_html_prefix='Leiden clusters UMAP SVG: '))
    

In [ ]:
if has_rna_file:
    leiden_df = pd.DataFrame([adata.obs['leiden'], adata.obs['samples']]).T.set_index('samples')
    clusters = list(leiden_df['leiden'].unique())
    display(leiden_df.value_counts().to_frame())
    display(Markdown(f"__Table {table_counter}__ Cluster distribution of samples."))


In [ ]:
if has_rna_file:
    display(Markdown(f"Based on the normalized most variable genes in RNA-seq data matrix {len(clusters)} clusters were identified (Fig. {fig_counter}). The number of samples in each cluster is shown in Table {table_counter}."))
    fig_counter += 1
    table_counter += 1

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Computing Differentially Expressed Genes
{% endif %}

In [ ]:
if has_rna_file:
    cluster_dges = {}
    dge_table = {}
    for c in sorted(clusters):
        cluster_dges[c] = {}
        cluster_samples = list(leiden_df[leiden_df['leiden'] == c].index.values)
        other_samples = [s for s in rna_df.columns if s not in cluster_samples]

        with suppress_output():
            dge = limma_voom_differential_expression(
                rna_df[other_samples], rna_df[cluster_samples],
                voom_design=True,
            )
        dge_filename = f'results/{rna_filename}_cluster_{c}_vs_rest_dge_limma.tsv'
        dge.sort_values(by="adj.P.Val", ascending=True).to_csv(dge_filename, sep='\t')
        sig_dge = dge[dge['adj.P.Val'] < 0.01]
        sig_up_gs = list(sig_dge[sig_dge['logFC'] > 0].index.values)
        sig_dn_gs = list(sig_dge[sig_dge['logFC'] < 0].index.values)
        if len(sig_up_gs) > 2000:
            sig_up_gs = sig_up_gs[:2000]
        elif len(sig_up_gs) < 50:
            sig_dge = dge[dge['adj.P.Val'] < 0.05]
            sig_up_gs = list(sig_dge[sig_dge['logFC'] > 0].index.values)
        if len(sig_dn_gs) > 2000:
            sig_dn_gs = sig_dn_gs[:2000]
        elif len(sig_dn_gs) < 50:
            sig_dge = dge[dge['adj.P.Val'] < 0.05]
            sig_dn_gs = list(sig_dge[sig_dge['logFC'] > 0].index.values)
        cluster_dges[c]['up'] = sig_up_gs
        cluster_dges[c]['down'] = sig_dn_gs
        dge_table[c] = {}
        dge_table[c]['up'] = len(cluster_dges[c]['up'])
        dge_table[c]['down'] = len(cluster_dges[c]['down'])
        display(FileLink(dge_filename, result_html_prefix=f'Differential expression for cluster {c}: '))

In [ ]:
if has_rna_file:
    enrichr_lists = {}
    for c in sorted(clusters):
        try:
            # filter for only protein coding genes to match Enrichr background.
            up_protein_coding = [g for g in cluster_dges[c]['up'] if lookup(g)]
            dn_protein_coding = [g for g in cluster_dges[c]['down'] if lookup(g)]
            up = enrichr_link_from_genes(up_protein_coding, f'Upregulated genes in cluster {c}')
            dn = enrichr_link_from_genes(dn_protein_coding, f'Downregulated genes in cluster {c}')
            dge_table[c]['Enrichr Up'] = up['link']
            dge_table[c]['Enrichr Down'] = dn['link']
            enrichr_lists[c] = (up, dn)
        except Exception as e:
            display(f'Error processing cluster {c}:', e)

In [ ]:
if has_rna_file:
    dge_table_df = pd.DataFrame(dge_table).T
    dge_table_df.index.name = 'Cluster'
    dge_table_df = dge_table_df.style.format({'Enrichr Up': lambda url: f'<a href="{url}" rel="noopener noreferrer" target="_blank">Enrichr - up genes</a>',
                                              'Enrichr Down': lambda url: f'<a href="{url}" rel="noopener noreferrer" target="_blank">Enrichr - down genes</a>'})
    display(dge_table_df)
    display(Markdown(f"__Table {table_counter}__ Number of upregulated and downregulated genes in each cluster."))
    table_counter += 1

In [ ]:
if has_rna_file:
    enrichr_labels = {}
    letter_counter = 0
    for cluster in enrichr_lists:
        enrichr_labels[cluster] = {}
        up, dn = enrichr_lists[cluster]
        display(Markdown(f'### Cluster {cluster}'))
        display(Markdown(f'#### Upregulated genes'))
        try:
            res_list = enrich_libraries(up['userListId'])
            enrichr_labels[cluster]['up'] = res_list
            fig = enrichr_figure(res_list)
            fig.savefig(f'figures/enrichr_up_cluster_{cluster}.png', bbox_inches='tight')
            fig.savefig(f'figures/enrichr_up_cluster_{cluster}.svg', bbox_inches='tight')
            display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Enrichr results for up-regulated genes in cluster {cluster} from GO BP, Wikipathways, and MGI. \* indicates adjusted p-value < 0.01."))
            display(FileLink(f'figures/enrichr_up_cluster_{cluster}.png', result_html_prefix='Enrichr figure for upregulated genes in cluster PNG: '))
            display(FileLink(f'figures/enrichr_up_cluster_{cluster}.svg', result_html_prefix='Enrichr figure for upregulated genes in cluster SVG: '))
            display(Markdown(f'#### Downregulated genes'))
            letter_counter += 1
        except Exception as e:
            print(f'Error processing cluster {cluster} upregulated genes:', e)
        try:
            res_list = enrich_libraries(dn['userListId'])
            enrichr_labels[cluster]['down'] = res_list
            fig = enrichr_figure(res_list)
            fig.savefig(f'figures/enrichr_dn_cluster_{cluster}.png', bbox_inches='tight')
            fig.savefig(f'figures/enrichr_dn_cluster_{cluster}.svg', bbox_inches='tight')
            display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Enrichr results for down-regulated genes in cluster {cluster} from GO BP, Wikipathways, and MGI. \* indicates adjusted p-value < 0.01."))
            display(FileLink(f'figures/enrichr_dn_cluster_{cluster}.png', result_html_prefix='Enrichr figure for downregulated genes in cluster PNG: '))
            display(FileLink(f'figures/enrichr_dn_cluster_{cluster}.svg', result_html_prefix='Enrichr figure for downregulated genes in cluster SVG: '))
            letter_counter += 1
        except Exception as e:
            print(f'Error processing cluster {cluster} downregulated genes:', e)
        
fig_counter += 1

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Consensus Cluster Labels
{% endif %}

In [ ]:
if has_rna_file:
    labels = label_clusters(enrichr_labels)
    discussion_results['Leiden Clusters Labeled with GPT from Enrichr Pathways & Phenotypes'] = convert_to_string(labels)
    adata.obs['label'] = adata.obs['leiden'].map(lambda c: labels.get(c, 'Unlabled') + f' (C{c})')
    cmap = plt.cm.get_cmap('tab20b', len(clusters))
    for i, cluster in enumerate(sorted(clusters)):
        mask = adata.obs['leiden'] == cluster
        ax = plt.scatter(x=adata.obsm['X_umap'][mask, 0], y=adata.obsm['X_umap'][mask, 1], 
                    color=cmap(i), label=f"{labels.get(cluster, 'Unlabeled')} (C{cluster})", s=10)
    plt.legend(title='Leiden clusters', bbox_to_anchor=(1.05, 1), loc='best')
    ax.axes.set_xticks([])
    ax.axes.set_yticks([])
    plt.xlabel('UMAP-1')
    plt.ylabel('UMAP-2')
    plt.savefig(f'figures/{rna_filename}_clusters_labeled.png', bbox_inches='tight')
    plt.savefig(f'figures/{rna_filename}_clusters_labeled.svg', bbox_inches='tight')

In [ ]:
if has_rna_file:
    display(Markdown(f"__Fig. {fig_counter}__ Consensus labels created using GPT-4 and significantly enriched terms for differentially expressed genes from leiden clusters."))
    display(FileLink(f'figures/{rna_filename}_clusters_labeled.png', result_html_prefix='Clusters labeled UMAP PNG: '))
    display(FileLink(f'figures/{rna_filename}_clusters_labeled.svg', result_html_prefix='Clusters labeled UMAP SVG: '))

In [ ]:
if has_rna_file:
    display(Markdown(f"Consensus labels were created for each cluster based on the most significantly enriched terms from the up- and down-regulated genes (Fig. {fig_counter -1}{ascii_uppercase[0]}-{ascii_uppercase[letter_counter]}) and are shown in Fig. {fig_counter}."))
    fig_counter += 1
    letter_counter = 0

In [ ]:
if has_rna_file:
    display(Markdown(describe_clusters(clean_enrichr_lables(enrichr_labels))))

In [ ]:
if has_rna_file:
    display(Markdown(f"*\*This section was generated with GPT-4 and should be interpreted with caution*\*"))

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Metadata Analysis
Using the uploaded metadata file, we can observe the relative occurrences of different metadata categories in the dataset per cluster.
{% endif %}

In [ ]:
if has_rna_file and has_meta_file:
    metadata_cluster_values = {}
    categorical_labels = set()
    if 'data_type' in meta_df.index:
        meta_df = meta_df.drop(index='data_type')
    for c in sorted(clusters):
        metadata_cluster_values[c] = []
        cluster_samples = list(set(leiden_df[leiden_df['leiden'] == c].index.values).intersection(meta_df.index))
        meta_cluster = meta_df.loc[cluster_samples]
        numeric_data = []
        categorical_data = []
        for col in meta_cluster.columns:
            try:
                numeric_data.append([col, meta_cluster[col].astype(float).mean().round(2), meta_cluster[col].astype(float).std().round(2)])
                metadata_cluster_values[c].append({'Attribute': col, 'Mean': meta_cluster[col].astype(float).mean(), 'Std': meta_cluster[col].astype(float).std()})
            except:
                categorical_labels.add(col)
                categorical_data.append([col, meta_cluster[col].value_counts()])
        numeric_data_df = pd.DataFrame(numeric_data, columns=['Attribute', 'Mean', 'Std'])
        if not numeric_data_df.empty:
            numeric_data= numeric_data_df.set_index('Attribute')
            display(numeric_data)
            display(Markdown(f"__Table {table_counter}__ Numeric metadata for cluster {c}"))
            table_counter += 1
        else:
            display(Markdown(f"No metadata was available for cluster {c} or an error occurred."))
        data = []
        for name, value_count in categorical_data:
            total_count = value_count.sum()
            for index, count in value_count.items():
                percentage = (count / total_count) * 100
                data.append({'Category': name, 'Value': index, 'Count': count, 'Percentage': percentage.round(1)})
        result_df = pd.DataFrame(data)
        if not result_df.empty:
            grouped_df = result_df.set_index(['Category', 'Value'])
            display(grouped_df)
            display(Markdown(f"__Table {table_counter}__ Categorical metadata for cluster {c}"))
            table_counter += 1
        else:
            display(Markdown(f"No metadata was available for cluster {c} or an error occurred."))

In [ ]:
if has_rna_file and has_meta_file:
    try:
        results = []
        n_pop = len(meta_df)
        for cluster_key, cluster_data in metadata_cluster_values.items():
            n_clus = len(leiden_df[leiden_df['leiden'] == cluster_key])
            for attribute in cluster_data:
                attr_name = attribute['Attribute']
                attr_mean = attribute['Mean']
                attr_std = attribute['Std']
                population_mean = meta_df[attr_name].astype(float).mean()
                population_std = meta_df[attr_name].astype(float).std()
            
                t_statistic, p_value = ttest_ind_from_stats(attr_mean, attr_std, n_clus, population_mean, population_std, n_pop, equal_var=False)
                if np.isnan(p_value):
                    p_value = 1
                    t_statistic = 0
                result = {
                    'Cluster': cluster_key,
                    'Metadata Attribute': attr_name,
                    'Cluster Mean': attr_mean,
                    'All Samples Mean': population_mean,
                    'T-Statistic': t_statistic,
                    'P-Value': p_value
                }
                results.append(result)

        # Create a DataFrame from the results list
        df_results = pd.DataFrame(results)

        _, pvalues_adj, _, _ = multipletests(df_results['P-Value'].values, method='fdr_bh')
        df_results['Adj. P-Value'] = pvalues_adj
        df_meta_sorted = df_results.set_index('Cluster').sort_values(['Cluster', 'Adj. P-Value'])
        df_meta_sorted_sig = df_meta_sorted[df_meta_sorted['Adj. P-Value'] < 0.05]
        if not df_meta_sorted_sig.empty:
            df_meta_sorted_sig.to_csv('results/sig_metadata_attrs_cluster.tsv', sep='\t')
            for cluster in metadata_cluster_values:
                if df_meta_sorted_sig[df_meta_sorted_sig.index == cluster].empty:
                    display(Markdown(f"No significant metadata attributes found for cluster {cluster}."))
                    continue
                display(df_meta_sorted_sig[df_meta_sorted_sig.index == cluster].head(10))
                display(Markdown(f"__Table {table_counter}__ Significant up and down attributes for numerical metadata for cluster {cluster}."))
                table_counter += 1
            display(FileLink('results/sig_metadata_attrs_cluster.tsv', result_html_prefix='Significant up and down attributes for numerical metadata for each cluster: '))
        else:
            display(df_meta_sorted)
            display(Markdown("No significant metadata attributes found."))
    except Exception as e:
        print('Error computing significance for metadata attributes in each cluster:', e)

In [ ]:
%%appyter code_exec
{% if rna_file.value != '' and membrane_screener_.raw_value %}
proteins = pd.read_csv('https://appyters.maayanlab.cloud/storage/Tumor_Gene_Target_Screener/surfaceome.csv')
membrane_proteins = list(proteins['genename'].map(lookup).dropna().values)
{% endif %}

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Cell Surface Target Identification
{% endif %}

In [ ]:
%%appyter code_exec

if has_rna_file:    
    targets = {}
    bgs = {'GTEx': 'gtex-gene-stats.tsv', 'ARCHS4': 'archs4-gene-stats.tsv', 'TS': 'ts_10x_cell-ontology-class_donors_tissue-labels_v1.tsv'}
    def find_targets(rna_df, bg, targets):
        df_bg_stats = pd.read_csv(f"https://appyters.maayanlab.cloud/storage/Tumor_Gene_Target_Screener/{bgs[bg]}", sep='\t', index_col=[0,1])
        df_bg_genes = df_bg_stats.unstack().index.map(lambda idx: lookup(idx.partition('.')[0]))
        df_bg_stats = df_bg_stats.unstack().groupby(df_bg_genes, observed=True).median().stack()
        df_bg_expr = df_bg_stats.loc[(slice(None), ['25%', '50%', '75%']), :].unstack()
        common_index = list(set(rna_df.index) & set(df_bg_expr.index))
        expr_df = rna_df.loc[common_index, :]
        expr_df.reset_index(inplace=True)
        expr_df.drop_duplicates(subset='gene_id', inplace=True)
        expr_df.set_index('gene_id', inplace=True, drop=True)
        {% if normalize_to_bg.raw_value %}
        target_distribution = df_bg_expr.loc[common_index, :].median(axis=1)
        df_expr_norm = qnorm.quantile_normalize(expr_df.loc[common_index, :], target=target_distribution)
        df_bg_expr_norm = qnorm.quantile_normalize(df_bg_expr.loc[common_index, :], target=target_distribution)
        {% else %}
        df_expr_norm = expr_df.loc[common_index, :]
        df_bg_expr_norm = df_bg_expr.loc[common_index, :]
        {% endif %}
        for cluster in tqdm(list(leiden_df['leiden'].unique())):
            if cluster not in targets: targets[cluster] = {}
            cluster_samples = list(leiden_df[leiden_df['leiden'] == cluster].index.values)
            with suppress_output():
                df_bg_expr_norm.columns = df_bg_expr_norm.columns.to_flat_index().map(lambda s: ', '.join(s))
                dge = limma_voom_differential_expression(
                    df_bg_expr_norm, df_expr_norm[cluster_samples],
                    voom_design=True,
                )
                targets[cluster][bg] = dge[(dge['adj.P.Val'] < 0.01) & (dge['t'] > 0)].sort_values('t', ascending=False).index.values 

In [ ]:
if has_rna_file: 
    print('Finding Targets using ARCHS4')
    find_targets(rna_df, 'ARCHS4', targets)

In [ ]:
if has_rna_file:    
    print('Finding Targets using GTEx')
    find_targets(rna_df, 'GTEx', targets)

In [ ]:
if has_rna_file:      
    print('Finding Targets using Tabula Sapiens')
    find_targets(rna_df, 'TS', targets)

In [ ]:
%%appyter code_eval

if has_rna_file:
    YlGnBu = cm.get_cmap('YlGnBu_r', 8)
    cmap = {"None":YlGnBu(0), "ARCHS4": YlGnBu(1), "GTEx":YlGnBu(2), "TS": YlGnBu(4), "ARCHS4-GTEx":YlGnBu(3),  "ARCHS4-TS": YlGnBu(5), "GTEx-TS": YlGnBu(6), "All": YlGnBu(7)}

    top_targets_n = 100
    target_list = []
    for cluster in targets:
        for bg in targets[cluster]:
            {% if rna_file.value != '' and membrane_screener_.raw_value %}
            target_list.extend(list(filter(lambda g: g in membrane_proteins, targets[cluster][bg][:top_targets_n])))
            {% else %}
            target_list.extend(list(targets[cluster][bg][:top_targets_n]))
            {% endif %}
    data1, data2, data3 = [], [], []
    similarity = []
    target_list = list(set(target_list))

    for gene in target_list:
        a = [1 if gene in targets[c]['ARCHS4'][:top_targets_n] else 0 for c in clusters]
        g = [2 if gene in targets[c]['GTEx'][:top_targets_n] else 0 for c in clusters]
        l = [4 if gene in targets[c]['TS'][:top_targets_n] else 0 for c in clusters]
        data1.append(a)
        data2.append(g)
        data3.append(l)
        similarity.append(np.dot(np.dot(np.array(a), np.array(g)), np.array(l)))

    data = np.add(np.add(data1, data2), data3)

    membrane_target_mat = pd.DataFrame(data)
    membrane_target_mat.columns = [f"Cluster {c}" for c in clusters]
    membrane_target_mat.index = target_list

    membrane_target_mat['count'] = membrane_target_mat.sum(axis=1)
    membrane_target_mat = membrane_target_mat[membrane_target_mat['count'] >= 7]
    membrane_target_mat = membrane_target_mat.rename_axis('Membrane Target').sort_values(by = ['count', 'Membrane Target'], ascending = [False, True]).drop('count', axis=1)

    h = membrane_target_mat.shape[0]

    cluster_targets= {}
    for col in membrane_target_mat.columns:
        cluster_targets[col + ' Cell Surface Target'] = ','.join(list(membrane_target_mat[col][membrane_target_mat[col] >= 5].index.values))

    g = sns.clustermap(membrane_target_mat, figsize=(4,0.3*h+2*(h<15)), cmap=YlGnBu, cbar_pos=None, dendrogram_ratio=0.1-(h<40)*0.01*(h-30), row_cluster=False, xticklabels=True, yticklabels=True)
    g.ax_row_dendrogram.legend(handles=[Rectangle((0, 0), 0, 0, color=val, label=key) for key, val in cmap.items()],
                                    title='Background', loc='upper right')

    plt.show()

    g.savefig(f'figures/{rna_filename}_membrane_target_matrix.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_membrane_target_matrix.svg', dpi=300)
    membrane_target_mat.to_csv(f'results/{rna_filename}_membrane_target_mat.csv')
    display(Markdown(f"__Fig. {fig_counter}__ Top 100 consensus targets for each cluster with adjusted p-value < 0.01 compared to ARCHS4, GTEx, and Tabula Sapiens (TS) healthy backgrounds with limma voom."))
    display(FileLink(f'results/{rna_filename}_membrane_target_mat.csv', result_html_prefix='Download Membrane Target Matrix: '))
    display(FileLink(f'figures/{rna_filename}_membrane_target_matrix.png', result_html_prefix='Membrane Target Matrix PNG: '))
    display(FileLink(f'figures/{rna_filename}_membrane_target_matrix.svg', result_html_prefix='Membrane Target Matrix SVG: '))

In [ ]:
if has_rna_file:
    display(Markdown(f"Cell surface targets were identified for each cluster and are ordered by the number of healthy backgrounds they were upregulated compared to (Fig. {fig_counter})."))
    fig_counter += 1

In [ ]:
if has_rna_file:
    display(Markdown(create_results_text(cluster_targets)))

In [ ]:
if has_rna_file:
    display(Markdown(f"*\*This section was generated with GPT-4 and should be interpreted with caution*\*"))

In [ ]:
if has_rna_file:
    rna_df_log2_zscore = zscore(log2_normalize(qnorm.quantile_normalize(rna_df)), axis=1)
    rna_df_log2_zscore

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Transcription Factor Enrichment Analysis with ChEA3
{% endif %}

In [ ]:
if has_rna_file:    
    chea_res = {}
    for patient in tqdm(rna_df_log2_zscore.columns.values, leave=False, mininterval=30):
        chea_res[patient] = {}
        down_rows = list(filter(lambda g: lookup(g) ,rna_df_log2_zscore[patient].astype(float).sort_values(ascending=True).index.values))[:chea3_n]
        up_rows = list(filter(lambda g: lookup(g) ,rna_df_log2_zscore[patient].astype(float).sort_values(ascending=False).index.values))[:chea3_n]

        chea_down = get_chea3_results(down_rows, f'{patient}_down_genes')
        chea_up = get_chea3_results(up_rows, f'{patient}_up_genes')
        chea_res[patient]['down'] = chea_down
        chea_res[patient]['up'] = chea_up

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
#### Full ChEA Heatmap - Up Genes
{% endif %}

In [ ]:
if has_rna_file:
    up_chea_comb = pd.concat([chea_res[p]['up'] for p in chea_res], axis=1)
    up_chea_comb.columns = list(chea_res.keys())
    try:
        g = create_annotated_clustermap(up_chea_comb, meta_df, leiden_df, default_meta_cols)
    except:
        plt.clf()
        g = create_clustermap(up_chea_comb.astype(float), cbar_label='Mean Rank', ylabel='Transcription Factors')
    clustergrammer_res = clustergrammer_link(up_chea_comb, f'{rna_filename}_chea_up_heatmap')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{rna_filename}_chea_up_heatmap.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_chea_up_heatmap.svg', dpi=300)

In [ ]:
if has_rna_file:
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Integrated Mean Rank from ChEA3 from up-regulated genes for all samples and all transcription factors based on z-scored expression."))
    letter_counter += 1
    display(FileLink(f'figures/{rna_filename}_chea_up_heatmap.png', result_html_prefix='ChEA3 Up Heatmap PNG: '))
    display(FileLink(f'figures/{rna_filename}_chea_up_heatmap.svg', result_html_prefix='ChEA3 Up Heatmap SVG: '))

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Top 20 Variable TFs - ChEA Heatmap (Up Genes)
{% endif %}

In [ ]:
if has_rna_file: 
    top_20_var_tfs = up_chea_comb.var(axis=1).sort_values(ascending=False).index[:20]
    try:
        g = create_annotated_clustermap(up_chea_comb.loc[top_20_var_tfs].astype(float), meta_df, leiden_df, default_meta_cols, cbar_label='Mean Rank', ylabel='Top 20 TFs by Variance', yticklabels=True)
    except:
        plt.clf()
        g = create_clustermap(up_chea_comb.loc[top_20_var_tfs].astype(float), ylabel='Top 20 TFs by Variance', cbar_label='Mean Rank', yticklabels=True)
    clustergrammer_res = clustergrammer_link(up_chea_comb.loc[top_20_var_tfs], f'{rna_filename}_chea_up_heatmap_top_var_tfs')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{rna_filename}_top_var_tfs_up_heatmap.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_top_var_tfs_up_heatmap.svg', dpi=300)

In [ ]:
if has_rna_file:    
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Integrated Mean Rank from ChEA3 from up-regulated genes for all samples and the top 20 most variable transcription factors based on z-scored expression."))
    letter_counter += 1
    display(FileLink(f'figures/{rna_filename}_top_var_tfs_up_heatmap.png', result_html_prefix='Most Variable TFs Heatmap (up genes) PNG: '))
    display(FileLink(f'figures/{rna_filename}_top_var_tfs_up_heatmap.svg', result_html_prefix='Most Variable TFs Heatmap (up genes) SVG: '))

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
#### Full ChEA Heatmap - Down Genes
{% endif %}

In [ ]:
if has_rna_file:    
    down_chea_comb = pd.concat([chea_res[p]['down'] for p in chea_res], axis=1)
    down_chea_comb.columns = list(chea_res.keys())
    try:
        g = create_annotated_clustermap(down_chea_comb.astype(float), meta_df, leiden_df, default_meta_cols)
    except:
        plt.clf()
        g = create_clustermap(down_chea_comb.astype(float), ylabel='Transcription Factors', cbar_label='Mean Rank')
    clustergrammer_res = clustergrammer_link(down_chea_comb, f'{rna_filename}_chea_dn_heatmap')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{rna_filename}_chea_dn_heatmap.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_chea_dn_heatmap.svg', dpi=300)

In [ ]:
if has_rna_file:    
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Integrated Mean Rank from ChEA3 from down-regulated genes for all samples and all transcription factors based on z-scored expression."))
    letter_counter += 1
    display(FileLink(f'figures/{rna_filename}_chea_dn_heatmap.png', result_html_prefix='ChEA3 Down Heatmap PNG: '))
    display(FileLink(f'figures/{rna_filename}_chea_dn_heatmap.svg', result_html_prefix='ChEA3 Down Heatmap SVG: ')) 

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Top 20 Variable TFs - ChEA Heatmap (Down Genes)
{% endif %}

In [ ]:
if has_rna_file:    
    top_20_var_tfs = down_chea_comb.var(axis=1).sort_values(ascending=False).index[:20]
    try:
        g = create_annotated_clustermap(down_chea_comb.loc[top_20_var_tfs].astype(float), meta_df, leiden_df, default_meta_cols, cbar_label='Mean Rank', ylabel='Top 20 TFs by Variance', yticklabels=True)
    except:
        plt.clf()
        g = create_clustermap(down_chea_comb.loc[top_20_var_tfs].astype(float), cbar_label='Mean Rank', ylabel='Top 20 TFs by Variance', yticklabels=True)
    clustergrammer_res = clustergrammer_link(down_chea_comb.loc[top_20_var_tfs].astype(float), f'{rna_filename}_chea_dn_heatmap_top_var_tfs')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{rna_filename}_top_var_tfs_dn_heatmap.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_top_var_tfs_dn_heatmap.svg', dpi=300)

In [ ]:
if has_rna_file:    
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Integrated Mean Rank from ChEA3 from down-regulated genes for all samples and the top 20 most variable transcription factors based on z-scored expression."))
    display(FileLink(f'figures/{rna_filename}_top_var_tfs_dn_heatmap.png', result_html_prefix='Most Variable TFs Heatmap (down genes) PNG: '))
    display(FileLink(f'figures/{rna_filename}_top_var_tfs_dn_heatmap.svg', result_html_prefix='Most Variable TFs Heatmap (down genes) SVG: '))

In [ ]:
if has_rna_file:
    display(Markdown(f"ChEA3 vectors for Z-scored gene expression for each sample is visualized for both the up- and down-regulated genes as well as for the top 20 TFs by variance (Fig. {fig_counter}{ascii_uppercase[0]}-{fig_counter}{ascii_uppercase[letter_counter]})."))
    letter_counter = 0
    fig_counter += 1

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Computing TF intermediates using Protein-Protein Co-Expression
{% endif %}

In [ ]:
if has_rna_file:    
    ids = list(chea_res.keys())
    n_TFs = 20
    def get_protein_coexpression():
        protein_coexp = pd.read_csv('https://minio.dev.maayanlab.cloud/x2k-tr/ccleproteomics_normalized_gene_correlation_matrix.txt.gz', sep='\t', compression='gzip', index_col=0)
        up_TFs = {}
        up_intermediates = {}
        dn_TFs = {}
        dn_intermediates = {}

        for id in tqdm(ids, leave=False, mininterval=30, desc="Augmenting TFs..."):
            up_TFs[id] = chea_res[id]['up'].astype(float).sort_values(by='Score', ascending=True).index[:n_TFs].to_list()
            up_intermediates[id] = augment_proteomics(gene_list=up_TFs[id], protein_coexp=protein_coexp)
            dn_TFs[id] = chea_res[id]['down'].astype(float).sort_values(by='Score', ascending=True).index[:n_TFs].to_list()
            dn_intermediates[id] = augment_proteomics(gene_list=dn_TFs[id], protein_coexp=protein_coexp)
        
        up_TFs = pd.DataFrame(up_TFs)
        up_intermediates = pd.DataFrame(up_intermediates)
        up_TFs.to_csv(f"results/up_TFs.csv", index=False)
        up_intermediates.to_csv(f"results/up_intermediates.csv", index=False)

        dn_TFs = pd.DataFrame(dn_TFs)
        dn_intermediates = pd.DataFrame(dn_intermediates)
        dn_TFs.to_csv(f"results/dn_TFs.csv", index=False)
        dn_intermediates.to_csv(f"results/dn_intermediates.csv", index=False)
        return up_TFs, up_intermediates, dn_TFs, dn_intermediates

In [ ]:
if has_rna_file:
    up_TFs, up_intermediates, dn_TFs, dn_intermediates = get_protein_coexpression()

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Inferring Kinases using KEA3 from TFs + Intermediates
{% endif %}

In [ ]:
if has_rna_file:    
    # Compute normalized kinase enrichment scores
    up_kinases = {}
    dn_kinases = {}
    x2k_up = {}
    x2k_dn = {}
    
    for id in tqdm(ids, leave=False, mininterval=30, desc="Computing RNA-seq derived KEA scores..."):
        up_geneset = list(set(list(up_TFs[id]) + list(filter(lambda x: x is not None, up_intermediates[id]))))
        dn_geneset = list(set(list(dn_TFs[id]) + list(filter(lambda x: x is not None, dn_intermediates[id]))))
        up_results = get_kea3_results(up_geneset)
        dn_results = get_kea3_results(dn_geneset)
        up_kinases[id] = up_results.astype(float).sort_values(by="Score", ascending=True).index.to_list()
        dn_kinases[id] = dn_results.astype(float).sort_values(by="Score", ascending=True).index.to_list()
        if len(x2k_up) == 0:
            KINASES = up_results.Score.index.to_list()
        x2k_up[id] = up_results.Score.loc[KINASES]
        x2k_dn[id] = dn_results.Score.loc[KINASES]


    pd.DataFrame(up_kinases).to_csv(f"results/up_kinases.csv", index=False)
    pd.DataFrame(dn_kinases).to_csv(f"results/dn_kinases.csv", index=False)
    x2k_up = pd.DataFrame(x2k_up, index=KINASES)
    x2k_dn = pd.DataFrame(x2k_dn, index=KINASES)
    x2k_up.to_csv(f"results/up_x2k_meanRank.csv")
    x2k_dn.to_csv(f"results/dn_x2k_meanRank.csv")

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
#### X2K Heatmap - Up Genes
{% endif %}

In [ ]:
if has_rna_file: 
    try:
        g = create_annotated_clustermap(x2k_up.astype(float), meta_df, leiden_df, default_meta_cols, cbar_label='Mean Rank', ylabel='Kinases', xlabel='Samples')
    except:
        plt.clf()
        g = create_clustermap(x2k_up.astype(float), cbar_label='Mean Rank', ylabel='Kinases', xlabel='Samples')
    clustergrammer_res = clustergrammer_link(x2k_up.astype(float), f'{rna_filename}_x2k_up')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{rna_filename}_x2k_up_heatmap.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_x2k_up_heatmap.svg', dpi=300)

In [ ]:
if has_rna_file:    
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Results of X2K (Expression2Kinases) pipeline for up-regulated genes visualized as hierarchically clustered heatmap for all kinases."))
    letter_counter += 1
    display(FileLink(f'figures/{rna_filename}_x2k_up_heatmap.png', result_html_prefix='X2K Up Heatmap PNG: '))
    display(FileLink(f'figures/{rna_filename}_x2k_up_heatmap.svg', result_html_prefix='X2K Up Heatmap SVG: '))

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
#### X2K Heatmap Top 20 Variable Kinases - Up Genes
{% endif %}

In [ ]:
if has_rna_file:    
    top_20_var_x2k = x2k_up.var(axis=1).sort_values(ascending=False).index[:20]
    try:
        g = create_annotated_clustermap(x2k_up.loc[top_20_var_x2k].astype(float), meta_df, leiden_df, default_meta_cols, cbar_label='Mean Rank', xlabel='Samples', ylabel='Top 20 Kinases by Variance', yticklabels=True)
    except:
        plt.clf()
        g = create_clustermap(x2k_up.loc[top_20_var_x2k].astype(float), cbar_label='Mean Rank', ylabel='Top 20 Kinases by Variance', xlabel='Samples', yticklabels=True)
    clustergrammer_res = clustergrammer_link(x2k_up.loc[top_20_var_x2k], f'{rna_filename}_x2k_up_top20')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{rna_filename}_x2k_up_top20_heatmap.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_x2k_up_top20_heatmap.svg', dpi=300)

In [ ]:
if has_rna_file:
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}.__ Results of X2K (Expression2Kinases) pipeline for up-regulated genes visualized as hierarchically clustered heatmap for 20 most variable kinases."))
    letter_counter += 1
    display(FileLink(f'figures/{rna_filename}_x2k_up_top20_heatmap.png', result_html_prefix='X2K Up Top 20 Kinases Heatmap PNG: '))
    display(FileLink(f'figures/{rna_filename}_x2k_up_top20_heatmap.svg', result_html_prefix='X2K Up Top 20 Kinases Heatmap SVG: ')) 

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
#### X2K Heatmap - Down Genes
{% endif %}

In [ ]:
if has_rna_file:   
    try: 
        g = create_annotated_clustermap(x2k_dn.astype(float), meta_df, leiden_df, default_meta_cols, cbar_label='Mean Rank', ylabel='Kinases', xlabel='Samples')
    except:
        plt.clf()
        g = create_clustermap(x2k_dn.astype(float), cbar_label='Mean Rank', ylabel='Kinases', xlabel='Samples')
    clustergrammer_res = clustergrammer_link(x2k_dn.astype(float), f'{rna_filename}_x2k_dn')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{rna_filename}_x2k_dn_heatmap.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_x2k_dn_heatmap.svg', dpi=300)

In [ ]:
if has_rna_file:
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Results of X2K (Expression2Kinases) pipeline for down-regulated genes visualized as hierarchically clustered heatmap for all kinases."))
    letter_counter += 1
    display(FileLink(f'figures/{rna_filename}_x2k_dn_heatmap.png', result_html_prefix='X2K Down Heatmap PNG: '))
    display(FileLink(f'figures/{rna_filename}_x2k_dn_heatmap.svg', result_html_prefix='X2K Down Heatmap SVG: '))

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
#### X2K Heatmap Top 20 Variable Kinases - Down Genes
{% endif %}

In [ ]:
if has_rna_file:    
    top_20_var_x2k = x2k_dn.var(axis=1).sort_values(ascending=False).index[:20]
    try:
        g = create_annotated_clustermap(x2k_dn.loc[top_20_var_x2k].astype(float), meta_df, leiden_df, default_meta_cols, cbar_label='Mean Rank', ylabel='Top 20 Kinases by Variance', xlabel='Samples', yticklabels=True)
    except:
        plt.clf()
        g = create_clustermap(x2k_dn.loc[top_20_var_x2k].astype(float), cbar_label='Mean Rank', ylabel='Top 20 Kinases by Variance', xlabel='Samples', yticklabels=True)
    clustergrammer_res = clustergrammer_link(x2k_dn.loc[top_20_var_x2k].astype(float), f'{rna_filename}_x2k_dn_top20')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{rna_filename}_x2k_dn_top20_heatmap.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_x2k_dn_top20_heatmap.svg', dpi=300)

In [ ]:
if has_rna_file:
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Results of X2K (Expression2Kinases) pipeline for down-regulated genes visualized as hierarchically clustered heatmap for 20 most variable kinases."))
    display(FileLink(f'figures/{rna_filename}_x2k_dn_top20_heatmap.png', result_html_prefix='X2K Down Top 20 Kinases Heatmap PNG: '))
    display(FileLink(f'figures/{rna_filename}_x2k_dn_top20_heatmap.svg', result_html_prefix='X2K Down Top 20 Kinases Heatmap SVG: '))

In [ ]:
if has_rna_file:
    display(Markdown(f"X2K results for each sample were visualized as hierarchically clustered heatmaps for all kinases and the top 20 most variable kinases up- and down-regulated genes (Fig. {fig_counter}{ascii_uppercase[0]}-{ascii_uppercase[letter_counter]})."))
    letter_counter = 0
    fig_counter += 1

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
### Visualize Common X2K Pathways
{% endif %}

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
#### Pathways inferred from up-genes:
{% endif %}

In [ ]:
if has_rna_file:
    pathway_results = {}
    rank_threshold = 30 # cut-off of top-ranked kinases considered as enriched in a sample
    n_edges = 30 # desired number of edges to show in the plot
    dir = "up" # enrichment direction to visualize
    kinase_col = "#FFD700" # kinase color
    tf_col = "#F08080" # TF color

    # Build network 
    edges_df = pd.DataFrame(columns=['target', 'source'])
    nodes_df = pd.DataFrame(columns=['id', 'type', 'color'])

    TFs = pd.read_csv(f"results/{dir}_TFs.csv")
    intermediates = pd.read_csv(f"results/{dir}_intermediates.csv")
    kinases = pd.read_csv(f"results/{dir}_kinases.csv")
    samples = kinases.columns.to_list()

    TRANSCRIPTION_FACTORS = TFs.to_numpy().flatten().tolist()
    KINASES = kinases.to_numpy().flatten().tolist()
    for cluster in list(sorted(leiden_df['leiden'].unique())):
        cluster_samples = list(leiden_df[leiden_df['leiden'] == cluster].index.values)
        for id in cluster_samples:
            new = pd.DataFrame([[tf, p] for p in intermediates[id].dropna() for tf in TFs[id].dropna()], columns=['target', 'source'])
            edges_df = pd.concat([edges_df, new])
            new = pd.DataFrame([[p, kinase] for kinase in kinases[id].dropna()[:rank_threshold] for p in intermediates[id].dropna()], columns=['target', 'source'])
            edges_df = pd.concat([edges_df, new])
        
        edges_df = edges_df.groupby(['target', 'source']).size().reset_index()
        edges_df.rename(columns={0: "count"}, inplace=True)
        edges_df['label'] = edges_df['count']
        edges_df = edges_df.loc[edges_df.source != edges_df.target]


        thresh = edges_df.sort_values(by='count', ascending=False).iloc[n_edges]['count']

        if (thresh * 4) <= len(cluster_samples):
            display(Markdown(f'No common pathways were identified for at least 25% of samples in cluster {cluster}.'))
            continue
        edges_show = edges_df.loc[edges_df['count'] >= thresh]
        
        nodes = pd.unique(edges_show[['target', 'source']].to_numpy().flatten().tolist())
        nodes = nodes[~pd.isnull(nodes)]
        nodes_df = pd.DataFrame({'id': nodes})
        nodes_df['type'] = 'intermediate'
        nodes_df['color'] = "#DCDCDC"
        nodes_df.loc[nodes_df['id'].isin(TRANSCRIPTION_FACTORS), 'type'] = 'TF'
        nodes_df.loc[nodes_df['id'].isin(KINASES), 'type'] = 'kinase'
        nodes_df.loc[nodes_df['type'] == 'TF', 'color'] = tf_col
        nodes_df.loc[nodes_df['type'] == 'kinase', 'color'] = kinase_col
        nodes_df['label'] = nodes_df['id']

        # Create a graph
        G = nx.Graph()

        # Add nodes
        for idx, row in nodes_df.iterrows():
            G.add_node(row['id'], group=row['type'], color=row['color'])

        # Add edges
        for idx, row in edges_show.iterrows():
            G.add_edge(row['source'], row['target'])

        # Manually set positions
        pos = {}
        group_positions = {'kinase': 0.8, 'intermediate': 0.5, 'TF': 0.2} 
        group_nodes = {group: [] for group in group_positions.keys()}
        for node, data in G.nodes(data=True):
            group_nodes[data['group']].append(node)
        for group, nodes in group_nodes.items():
            x_positions = np.linspace(0, 1, len(nodes))
            for x, node in zip(x_positions, nodes):
                pos[node] = (x, group_positions[group])

        nodes_data =G.nodes(data=True)
        clus_pathway =  {'kinases':[], 'intermediates': [], 'TFs': []}
        for n in nodes_data:
            if n[1]['group'] == 'kinase':
                clus_pathway['kinases'].append(n[0])
            elif n[1]['group'] == 'intermediate':
                clus_pathway['intermediates'].append(n[0])
            elif n[1]['group'] == 'TF':
                clus_pathway['TFs'].append(n[0])
        pathway_results[f"Common X2K Pathways - Upregulated Genes - Cluster {cluster}"] = convert_to_string(clus_pathway)
    
        node_colors = [G.nodes[n]['color'] for n in G.nodes()]
        if len(G.nodes()) > 60:
            display(Markdown(f'Too many nodes were identified for {cluster}.'))
            continue

        if len(G.nodes()) >= 50:
            node_size = 600
            font_size = 4
        elif len(G.nodes()) > 40:
            node_size = 800
            font_size = 6
        else:
            node_size = 1500
            font_size = 8
        plt.figure(figsize=(12, 5))
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_size)
        nx.draw_networkx_edges(G, pos, edgelist=edges_show.to_records(index=False), width=1)
        nx.draw_networkx_labels(G, pos, font_size=font_size, font_family='sans-serif')
        plt.savefig(f'figures/{dir}_x2k_pathways_C{cluster}.png', dpi=300)
        plt.savefig(f'figures/{dir}_x2k_pathways_C{cluster}.svg', dpi=300)
        nx.write_adjlist(G, f'results/{dir}_x2k_pathways_C{cluster}.adjlist')
        plt.show()
        display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Visualization of pathways commonly identified from up-genes from cluster {cluster}."))
        letter_counter += 1
        display(FileLink(f'figures/{dir}_x2k_pathways_C{cluster}.png', result_html_prefix='Pathways Inferred from Up-Genes PNG: '))
        display(FileLink(f'figures/{dir}_x2k_pathways_C{cluster}.svg', result_html_prefix='Pathways Inferred from Up-Genes SVG: '))
        display(FileLink(f'results/{dir}_x2k_pathways_C{cluster}.adjlist', result_html_prefix='Download Adjacency List: '))

In [ ]:
if has_rna_file:
    display(Markdown(f"Common X2K pathways were inferred from up-regulated genes for each cluster and visualized as network graphs (Fig. {fig_counter}{ascii_uppercase[0]}-{ascii_uppercase[letter_counter - 1]})."))
    fig_counter += 1
    letter_counter = 0

In [ ]:
%%appyter markdown
{% if rna_file.value != '' %}
#### Pathways inferred from down-genes:
{% endif %}

In [ ]:
if has_rna_file:
    rank_threshold = 30 # cut-off of top-ranked kinases considered as enriched in a sample
    n_edges = 30 # desired number of edges to show in the plot
    dir = "dn" # enrichment direction to visualize
    kinase_col = "#FFD700" # kinase color
    tf_col = "#F08080" # TF color

    # Build network 
    edges_df = pd.DataFrame(columns=['target', 'source'])
    nodes_df = pd.DataFrame(columns=['id', 'type', 'color'])

    TFs = pd.read_csv(f"results/{dir}_TFs.csv")
    intermediates = pd.read_csv(f"results/{dir}_intermediates.csv")
    kinases = pd.read_csv(f"results/{dir}_kinases.csv")
    samples = kinases.columns.to_list()

    TRANSCRIPTION_FACTORS = TFs.to_numpy().flatten().tolist()
    KINASES = kinases.to_numpy().flatten().tolist()
    for cluster in list(sorted(leiden_df['leiden'].unique())):
        cluster_samples = list(leiden_df[leiden_df['leiden'] == cluster].index.values)
        for id in cluster_samples:
            new = pd.DataFrame([[tf, p] for p in intermediates[id].dropna() for tf in TFs[id].dropna()], columns=['target', 'source'])
            edges_df = pd.concat([edges_df, new])
            new = pd.DataFrame([[p, kinase] for kinase in kinases[id].dropna()[:rank_threshold] for p in intermediates[id].dropna()], columns=['target', 'source'])
            edges_df = pd.concat([edges_df, new])

        edges_df = edges_df.groupby(['target', 'source']).size().reset_index()
        edges_df.rename(columns={0: "count"}, inplace=True)
        edges_df['label'] = edges_df['count']
        edges_df = edges_df.loc[edges_df.source != edges_df.target]

        thresh = edges_df.sort_values(by='count', ascending=False).iloc[n_edges]['count']
        if (thresh * 4) <= len(cluster_samples):
            display(Markdown(f'No common pathways were identified for at least 25% of samples in cluster {cluster}.'))
            continue
        edges_show = edges_df.loc[edges_df['count'] >= thresh]
        
        nodes = pd.unique(edges_show[['target', 'source']].to_numpy().flatten().tolist())
        nodes = nodes[~pd.isnull(nodes)]
        nodes_df = pd.DataFrame({'id': nodes})
        nodes_df['type'] = 'intermediate'
        nodes_df['color'] = "#DCDCDC"
        nodes_df.loc[nodes_df['id'].isin(TRANSCRIPTION_FACTORS), 'type'] = 'TF'
        nodes_df.loc[nodes_df['id'].isin(KINASES), 'type'] = 'kinase'
        nodes_df.loc[nodes_df['type'] == 'TF', 'color'] = tf_col
        nodes_df.loc[nodes_df['type'] == 'kinase', 'color'] = kinase_col
        nodes_df['label'] = nodes_df['id']

        # Create a graph
        G = nx.Graph()

        # Add nodes
        for idx, row in nodes_df.iterrows():
            G.add_node(row['id'], group=row['type'], color=row['color'])

        # Add edges
        for idx, row in edges_show.iterrows():
            G.add_edge(row['source'], row['target'])

        # Manually set positions
        pos = {}
        group_positions = {'kinase': 0.8, 'intermediate': 0.5, 'TF': 0.2} 
        group_nodes = {group: [] for group in group_positions.keys()}
        for node, data in G.nodes(data=True):
            group_nodes[data['group']].append(node)
        for group, nodes in group_nodes.items():
            x_positions = np.linspace(0, 1, len(nodes))
            for x, node in zip(x_positions, nodes):
                pos[node] = (x, group_positions[group])

        nodes_data =G.nodes(data=True)
        clus_pathway =  {'kinases':[], 'intermediates': [], 'TFs': []}
        for n in nodes_data:
            if n[1]['group'] == 'kinase':
                clus_pathway['kinases'].append(n[0])
            elif n[1]['group'] == 'intermediate':
                clus_pathway['intermediates'].append(n[0])
            elif n[1]['group'] == 'TF':
                clus_pathway['TFs'].append(n[0])
        pathway_results[f"Common X2K Pathways - Downregulated Genes - Cluster {cluster}"] = convert_to_string(clus_pathway)
        
        node_colors = [G.nodes[n]['color'] for n in G.nodes()]

        if len(G.nodes()) > 60:
            display(Markdown(f'Too many nodes were identified for {cluster}.'))
            continue

        if len(G.nodes()) > 50:
            node_size = 600
            font_size = 4
        elif len(G.nodes()) > 40:
            node_size = 400
            font_size = 4
        else:
            node_size = 1500
            font_size = 8
        plt.figure(figsize=(12, 5))
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_size)
        nx.draw_networkx_edges(G, pos, edgelist=edges_show.to_records(index=False), width=1)
        nx.draw_networkx_labels(G, pos, font_size=font_size, font_family='sans-serif') 
        plt.savefig(f'figures/{dir}_x2k_pathways_C{cluster}.png', dpi=300)
        plt.savefig(f'figures/{dir}_x2k_pathways_C{cluster}.svg', dpi=300)
        nx.write_adjlist(G, f'results/{dir}_x2k_pathways_C{cluster}.adjlist')
        plt.show()
        display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Visualization of pathways commonly identified from up-genes from cluster {cluster}."))
        letter_counter += 1
        display(FileLink(f'figures/{dir}_x2k_pathways_C{cluster}.png', result_html_prefix='Pathways Inferred from Down-Genes PNG: '))
        display(FileLink(f'figures/{dir}_x2k_pathways_C{cluster}.svg', result_html_prefix='Pathways Inferred from Down-Genes SVG: '))
        display(FileLink(f'results/{dir}_x2k_pathways_C{cluster}.adjlist', result_html_prefix='Download Adjacency List: '))


In [ ]:
if has_rna_file:
    display(Markdown(f"Common X2K pathways were inferred from down-regulated genes for each cluster and visualized as network graphs (Fig. {fig_counter}{ascii_uppercase[0]}-{ascii_uppercase[letter_counter -1]})."))
    fig_counter += 1
    letter_counter = 0

In [ ]:
if has_rna_file:
    display(Markdown(create_results_text_prompt(pathway_results, 'These are common pathways identified with X2K(RNA-seq expression to kinases) for up and down regulated genes per cluster. Please write in paragraph form')))

In [ ]:
if has_rna_file:
    display(Markdown(f"*\*This section was generated with GPT-4 and should be interpreted with caution*\*"))

In [ ]:
%%appyter markdown
{% if rna_file.value != '' and tumor_validate != [] %}
### Validation of Cancer Targets
{% endif %}

In [ ]:
if has_rna_file and len(tumor_validate) > 0:
    depmap = pd.read_csv('https://minio.dev.maayanlab.cloud/x2k-tr/DepMap_CRISPRGeneEffect.csv.gz', index_col=[0,1], compression='gzip')
    depmap_ct = depmap[depmap.index.get_level_values('OncotreePrimaryDisease').isin(tumor_validate)].mean().to_frame('Mean Gene Effect')

In [ ]:
if has_rna_file and len(tumor_validate) > 0:
    # DepMap Gene Effect scores
    flat_list = []
    for cluster, bgs in targets.items():
        for bg, bg_targets in bgs.items():
            for target in bg_targets[:10]:
                flat_list.append((target, (cluster, bg)))
    df = pd.DataFrame(flat_list, columns=['Target', f'Ranked top 10 in (cluster, background)'])
    df = df.groupby('Target').agg(list)
    df = pd.merge(df, depmap_ct, left_index=True, right_index=True, how='left')
    # Literature occurrences with 'cancer'
    generic_occ = []
    for target, row in df.iterrows():
        generic_occ.append(get_co_occurrence_count([target, 'cancer']))
    df['Literature co-occurrence with cancer'] = generic_occ

    # Literature occurrences with specific disease
    for tumortype in tumor_validate:
        specific_occ = []
        for target, row in df.iterrows():
            specific_occ.append(get_co_occurrence_count([target, tumortype]))
        df[f'Literature co-occurrence with {tumortype}'] = specific_occ

    display(df)
    display(Markdown(f"__Table {table_counter}__ Top 10 targets from each cluster and background with DepMap gene effect scores and literature co-occurrences with cancer and specific tumor types."))
    df.to_csv('results/target_validation.csv')
    display(FileLink(f"results/target_validation.csv", result_html_prefix=' Download DepMap and Literature Validation of Gene Targets:'))
    table_counter += 1

In [ ]:
if has_rna_file and len(tumor_validate) > 0:
    # DepMap Gene Effect scores
    up_df = x2k_up.mean(axis=1).to_frame('Up-Genes MeanRank')
    dn_df = x2k_dn.mean(axis=1).to_frame('Dn-Genes MeanRank')
    df = pd.concat([up_df, dn_df], axis=1, ignore_index=False)
    top_n_up = up_df.sort_values(by='Up-Genes MeanRank').index[:10]
    top_n_dn = dn_df.sort_values(by='Dn-Genes MeanRank').index[:10]
    df = df.loc[top_n_up.union(top_n_dn)]
    df = pd.merge(df, depmap_ct, left_index=True, right_index=True, how='left')
    # Literature occurrences with 'cancer'
    generic_occ = []
    for kinase, row in df.iterrows():
        generic_occ.append(get_co_occurrence_count([kinase, 'cancer']))
    df['Literature co-occurrence with cancer'] = generic_occ
    # Literature occurrences with specific disease
    for tumortype in tumor_validate:
        specific_occ = []
        for kinase, row in df.iterrows():
            specific_occ.append(get_co_occurrence_count([kinase, tumortype]))
        df[f'Literature co-occurrence with {tumortype}'] = specific_occ
    display(df)
    display(Markdown(f"__Table {table_counter}__ DepMap gene effect scores and literature co-occurrences with cancer and specific tumor types for top 10 kinases identified through either the up-regulated and down-regulated genes."))
    df.to_csv('results/kinase_validation.csv')
    display(FileLink(f"results/kinase_validation.csv", result_html_prefix=' Download DepMap and Literature Validation of Kinases:'))
    table_counter += 1

In [ ]:
%%appyter code_exec
{% if phospho_file.value != '' and impute_phospho_expr.raw_value %}
if has_phospho_file:
    percentages = []
    idx = []
    i = 0
    for _, row in tqdm(phospho_df.iterrows(), leave=False, mininterval=30, total=len(phospho_df)):
        value_counts = row.astype(float).isna().value_counts()
        pr = value_counts[0]/len(row)
        if pr > percent_rows_phospho:
            idx.append(i)
        percentages.append(pr)
        i += 1
    ax = pd.Series(percentages).hist()
    ax.set_ylabel("# Phosphosites")
    ax.set_xlabel("% with values")
{% endif %}

In [ ]:
%%appyter code_exec
{% if phospho_file.value != '' and impute_phospho_expr.raw_value %}
if has_phospho_file:
    display(Markdown(f"__Fig. {fig_counter}__ Distribution of missing values in phosphoproteomic data."))
    fig_counter += 1
{% endif %}

In [ ]:
%%appyter code_exec
if has_phospho_file:
    phospho_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    {% if phospho_file.value != '' and impute_phospho_expr.raw_value %}
    n1 = len(phospho_df)
    phospho_df_imputed = phospho_df.iloc[idx].T.fillna(phospho_df.iloc[idx].mean(axis=1)).T.dropna()
    zscored_phospho = zscore(phospho_df_imputed, axis=1)
    zscored_samps = zscore(phospho_df_imputed, axis=0)
    display(Markdown(f"Dropped {n1 - len(phospho_df_imputed)} rows with missing values. The distribution of missing values in phosphoproteomic data was visualized in Figure {fig_counter - 1}."))
    {% elif phospho_file.value != '' and not impute_phospho_expr.raw_value %}
    phospho_df.dropna(inplace=True)
    zscored_phospho = zscore(phospho_df, axis=1)
    zscored_samps = zscore(phospho_df, axis=0)
    {% endif %}

In [ ]:
%%appyter markdown
{% if phospho_file.value != '' %}
#### Most Variable Phosphosites
{% endif %}

In [ ]:
if has_phospho_file:
    zscored_phospho_var = zscored_samps.var(axis=1).sort_values(ascending=False)
    zscored_phospho_var_list = zscored_phospho_var.index[:100].tolist()
    df = pd.DataFrame(zscored_phospho_var, columns=['Variance'])
    df.index.names = ['Phosphosite']
    if '.' in zscored_phospho_var.index.values[0]:
        df['Gene Symbol'] = df.index.map(lambda x: lookup(x.split('.')[0]) if lookup(x.split('.')[0]) else x.split('.')[0])
    elif '_' in zscored_phospho_var.index.values[0]:
        df['Gene Symbol'] = df.index.map(lambda x: lookup(x.split('_')[0]) if lookup(x.split('_')[0]) else x.split('_')[0])
    else:
        df['Gene Symbol'] = df.index.map(lambda x: lookup(x) if lookup(x) else x)
    display(df.head(10))
    display(Markdown(f"__Table {table_counter}__ Top 10 most variable phosphosites."))
    table_counter += 1

In [ ]:
if has_phospho_file:    
    try:
        g = create_annotated_clustermap(zscored_phospho.loc[zscored_phospho_var_list], meta_df, leiden_df, default_meta_cols, cbar_label='Z-Score', ylabel='Samples', xlabel='Phosphosites')
    except:
        plt.clf()
        g = create_clustermap(zscored_phospho.loc[zscored_phospho_var_list], cbar_label='Z-Score', ylabel='Samples', xlabel='Phosphosites')
    clustergrammer_res = clustergrammer_link(zscored_phospho.loc[zscored_phospho_var_list], f'{phospho_filename}_top_var_phosphosites')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    ax = g.ax_heatmap
    ax.set_ylabel('Top 100 Phosphosites by Variance')
    ax.set_xlabel('Samples')
    g.savefig(f'figures/{phospho_filename}_top_var_phosphosites_heatmap.png', dpi=300)
    g.savefig(f'figures/{phospho_filename}_top_var_phosphosites_heatmap.svg', dpi=300)
    

In [ ]:
if has_phospho_file:
    display(Markdown(f"__Fig. {fig_counter}__ Hierarchically clustered heatmap of the top 100 most variable phosphosites."))
    display(FileLink(f'figures/{phospho_filename}_top_var_phosphosites_heatmap.png', result_html_prefix='Most Variable Phosphosites Heatmap PNG: '))
    display(FileLink(f'figures/{phospho_filename}_top_var_phosphosites_heatmap.svg', result_html_prefix='Most Variable Phosphosites Heatmap SVG: '))

In [ ]:
if has_phospho_file:
    display(Markdown(f"The top 10 most variable phosphosites were visualized as a hierarchically clustered heatmap (Fig. {fig_counter})."))
    fig_counter += 1

In [ ]:
if has_rna_file and has_phospho_file:  
    adata = sc.AnnData(zscored_samps.T.values, dtype="float")
    adata.var['phosphosite_names'] = zscored_samps.index.values
    adata.obs['samples'] = zscored_samps.columns.values
    adata.var['var_rank'] = (-np.var(adata.X, axis=0, dtype="float")).argsort()
    adata = adata[:, adata.var.var_rank < 5000]

    adata.obs['leiden'] = adata.obs['samples'].map(lambda s: leiden_df.loc[s]['leiden'] if s in leiden_df.index.values else 'N/A' )
    sc.pp.pca(adata)
    sc.pp.neighbors(adata, n_neighbors=n_neighbors)  # create neighborhood graph
    sc.tl.umap(adata, min_dist=min_dist, alpha=0.1) 
    clusters = adata.obs['leiden'].unique()
    cmap = plt.cm.get_cmap('tab20b', len(clusters))
    for i, cluster in enumerate(sorted(clusters)):
        mask = adata.obs['leiden'] == cluster
        ax = plt.scatter(x=adata.obsm['X_umap'][mask, 0], y=adata.obsm['X_umap'][mask, 1], 
                    color=cmap(i), label=f'Cluster {cluster}', s=10)
    plt.legend(title='Leiden clusters', bbox_to_anchor=(1.05, 1), loc='best')
    ax.axes.set_xticks([])
    ax.axes.set_yticks([])
    plt.xlabel('UMAP-1')
    plt.ylabel('UMAP-2')
    plt.savefig(f'figures/{phospho_filename}_umap_leiden_rna_clusters.png', dpi=300, bbox_inches='tight')
    plt.savefig(f'figures/{phospho_filename}_umap_leiden_rna_clusters.svg', dpi=300, bbox_inches='tight')
elif has_phospho_file:
    adata = sc.AnnData(zscored_samps.T.values, dtype="int")
    adata.var['protein_names'] = zscored_samps.index.values
    adata.obs['samples'] = zscored_samps.columns.values
    # Sort proteins by variance
    adata.var['var_rank'] = (-np.var(adata.X, axis=0, dtype="float")).argsort()
    adata = adata[:, adata.var.var_rank < 5000]
    # UMAP
    sc.pp.pca(adata)
    sc.pp.neighbors(adata, n_neighbors=n_neighbors)  # create neighborhood graph
    sc.tl.umap(adata, min_dist=min_dist, alpha=0.1)  # embed umap based on neighborhood graph
    sc.tl.leiden(adata, resolution=resolution)  # clustering
    clusters = adata.obs['leiden'].unique()
    cmap = plt.cm.get_cmap('tab20b', len(clusters))
    for i, cluster in enumerate(sorted(clusters)):
        mask = adata.obs['leiden'] == cluster
        ax = plt.scatter(x=adata.obsm['X_umap'][mask, 0], y=adata.obsm['X_umap'][mask, 1], 
                    color=cmap(i), label=f'Cluster {cluster}', s=10)
    plt.legend(title='Leiden clusters', bbox_to_anchor=(1.05, 1), loc='best')
    ax.axes.set_xticks([])
    ax.axes.set_yticks([])
    plt.xlabel('UMAP-1')
    plt.ylabel('UMAP-2')
    plt.savefig(f'figures/{phospho_filename}_umap_leiden.png', dpi=300, bbox_inches='tight')
    plt.savefig(f'figures/{phospho_filename}_umap_leiden.svg', dpi=300, bbox_inches='tight')

In [ ]:
if has_phospho_file and has_rna_file:
    display(Markdown(f"__Fig. {fig_counter}__ UMAP visualization of top 5000 most variable phosphosites colored by leiden clusters determined from RNA-seq expression."))
    display(FileLink(f'figures/{phospho_filename}_umap_leiden_rna_clusters.png', result_html_prefix='Phosphosite UMAP colored by RNA Leiden clusters PNG: '))
    display(FileLink(f'figures/{phospho_filename}_umap_leiden_rna_clusters.svg', result_html_prefix='Phosphosite UMAP colored by RNA Leiden clusters SVG: '))
elif has_phospho_file:
    display(Markdown(f"__Fig. {fig_counter}__ Leiden clusters determined from Z-scored phosphosites for 5000 most variable proteins."))
    display(FileLink(f'figures/{phospho_filename}_umap_leiden.png', result_html_prefix='Leiden clusters UMAP PNG: '))
    display(FileLink(f'figures/{phospho_filename}_umap_leiden.svg', result_html_prefix='Leiden clusters UMAP SVG: '))

In [ ]:
if has_phospho_file and has_rna_file:
    display(Markdown(f"A UMAP visualization was created of top 5000 most variable phosphosites colored by leiden clusters determined from RNA-seq expression (Fig. {fig_counter})."))
    fig_counter += 1
elif has_phospho_file:
    display(Markdown(f"A UMAP visualization was created of top 5000 most variable phosphosites colored by leiden clusters (Fig. {fig_counter})."))
    fig_counter += 1

In [ ]:
if has_phospho_file:
    kea_res = {}
    for patient in tqdm(zscored_phospho.columns, leave=False, mininterval=30, desc='Computing KEA3 Scores'):
        kea_res[patient] = {}
        if '.' in zscored_phospho.index.values[0]:
            down_rows = list(map(lambda g: lookup(g.split('.')[0]), filter(lambda g: lookup(g.split('.')[0]) , zscored_samps[patient].astype(float).sort_values(ascending=True).index.values)))[:kea3_n]
            up_rows = list(map(lambda g: lookup(g.split('.')[0]), filter(lambda g: lookup(g.split('.')[0]) , zscored_samps[patient].astype(float).sort_values(ascending=False).index.values)))[:kea3_n]
        elif '_' in zscored_phospho.index.values[0]:
            down_rows = list(map(lambda g: lookup(g.split('_')[0]), filter(lambda g: lookup(g.split('_')[0]) , zscored_samps[patient].astype(float).sort_values(ascending=True).index.values)))[:kea3_n]
            up_rows = list(map(lambda g: lookup(g.split('_')[0]), filter(lambda g: lookup(g.split('_')[0]) , zscored_samps[patient].astype(float).sort_values(ascending=False).index.values)))[:kea3_n]
        else:
            down_rows = list(map(lambda g: lookup(g.split('_')[0]), filter(lambda g: lookup(g) , zscored_samps[patient].astype(float).sort_values(ascending=True).index.values)))[:kea3_n]
            up_rows = list(map(lambda g: lookup(g.split('_')[0]), filter(lambda g: lookup(g) , zscored_samps[patient].astype(float).sort_values(ascending=False).index.values)))[:kea3_n]
        
        up_kinases = get_kea3_results(down_rows, f'{patient}_up_phosphosites')
        down_kinases = get_kea3_results(up_rows, f'{patient}_down_phosphosites')
        kea_res[patient]['up'] = up_kinases
        kea_res[patient]['down'] = down_kinases

In [ ]:
%%appyter markdown
{% if phospho_file.value != '' %}
#### Full KEA3 Heatmap - Up Phosphosites
{% endif %}

In [ ]:
if has_phospho_file:    
    up_kea_comb = pd.concat([kea_res[p]['up'] for p in kea_res], axis=1)
    up_kea_comb.columns = list(kea_res.keys())
    try:
        g = create_annotated_clustermap(up_kea_comb.astype(float), meta_df, leiden_df, default_meta_cols, cbar_label='Mean Rank', ylabel='Kinases', xlabel='Samples')
    except:
        g = create_clustermap(up_kea_comb.astype(float), cbar_label='Mean Rank', ylabel='Kinases', xlabel='Samples')
    clustergrammer_res = clustergrammer_link(up_kea_comb.astype(float), f'{phospho_filename}_kea_up')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{phospho_filename}_kea_up_heatmap.png', dpi=300)
    g.savefig(f'figures/{phospho_filename}_kea_up_heatmap.svg', dpi=300)

In [ ]:
if has_phospho_file:
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Integrated Mean Rank from KEA3 from up-regulated phosphosites for all samples and all kinases based on z-scored phosphorylation."))
    letter_counter += 1
    display(FileLink(f'figures/{phospho_filename}_kea_up_heatmap.png', result_html_prefix='KEA3 Up Heatmap PNG: '))
    display(FileLink(f'figures/{phospho_filename}_kea_up_heatmap.svg', result_html_prefix='KEA3 Up Heatmap SVG: '))

In [ ]:
%%appyter markdown
{% if phospho_file.value != '' %}
#### KEA3 Heatmap Top 20 Variable Kinases - Up Phosphosites
{% endif %}

In [ ]:
if has_phospho_file:   
    top_20_var_kinases = up_kea_comb.var(axis=1).sort_values(ascending=False).index[0:20]
    try:
        g = create_annotated_clustermap(up_kea_comb.loc[top_20_var_kinases].astype(float), meta_df, leiden_df, default_meta_cols, cbar_label='Mean Rank', ylabel='Top 20 Kinases by Variance', yticklabels=True)
    except:
        plt.clf()
        g = create_clustermap(up_kea_comb.loc[top_20_var_kinases].astype(float), cbar_label='Mean Rank', ylabel='Top 20 Kinases by Variance', yticklabels=True)
    clustergrammer_res = clustergrammer_link(up_kea_comb.loc[top_20_var_kinases].astype(float), f'{phospho_filename}_kea_up_top20')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{phospho_filename}_kea_up_top_var_heatmap.png', dpi=300)
    g.savefig(f'figures/{phospho_filename}_kea_up_top_var_heatmap.svg', dpi=300)

In [ ]:
if has_phospho_file:
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Integrated Mean Rank from KEA3 from up-regulated phosphosites for all samples and top 20 most variable kinases based on z-scored phosphorylation."))
    letter_counter += 1
    display(FileLink(f'figures/{phospho_filename}_kea_up_top_var_heatmap.png', result_html_prefix='KEA3 Up Top 20 Variable Kinases Heatmap PNG: '))
    display(FileLink(f'figures/{phospho_filename}_kea_up_top_var_heatmap.svg', result_html_prefix='KEA3 Up Top 20 Variable Kinases Heatmap SVG: '))

In [ ]:
%%appyter markdown
{% if phospho_file.value != '' %}
#### Full KEA3 Heatmap - Down Phosphosites
{% endif %}

In [ ]:
if has_phospho_file:    
    dn_kea_comb = pd.concat([kea_res[p]['down'] for p in kea_res], axis=1)
    dn_kea_comb.columns = list(kea_res.keys())
    try:
        g = create_annotated_clustermap(dn_kea_comb.astype(float), meta_df, leiden_df, default_meta_cols, cbar_label='Mean Rank', ylabel='Kinases', xlabel='Samples')
    except:
        plt.clf()
        g = create_clustermap(dn_kea_comb.astype(float), cbar_label='Mean Rank', ylabel='Kinases', xlabel='Samples')
    clustergrammer_res = clustergrammer_link(dn_kea_comb.astype(float), f'{phospho_filename}_kea_dn')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{phospho_filename}_kea_dn_heatmap.png', dpi=300)
    g.savefig(f'figures/{phospho_filename}_kea_dn_heatmap.svg', dpi=300)

In [ ]:
if has_phospho_file:    
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Integrated Mean Rank from KEA3 from down-regulated phosphosites for all samples and all kinases based on z-scored phosphorylation."))
    letter_counter += 1
    display(FileLink(f'figures/{phospho_filename}_kea_dn_heatmap.png', result_html_prefix='KEA3 Down Heatmap PNG: '))
    display(FileLink(f'figures/{phospho_filename}_kea_dn_heatmap.svg', result_html_prefix='KEA3 Down Heatmap SVG: '))

In [ ]:
%%appyter markdown
{% if phospho_file.value != '' %}
#### KEA3 Heatmap Top 20 Variable Kinases - Down Phosphosites
{% endif %}

In [ ]:
if has_phospho_file:
    top_20_var_kinases = dn_kea_comb.var(axis=1).sort_values(ascending=False).index[0:20]
    try:
        g = create_annotated_clustermap(dn_kea_comb.loc[top_20_var_kinases].astype(float), meta_df, leiden_df, default_meta_cols, cbar_label='Mean Rank', ylabel='Top 20 Kinases by Variance', yticklabels=True)
    except:
        plt.clf()
        g = create_clustermap(dn_kea_comb.loc[top_20_var_kinases].astype(float), cbar_label='Mean Rank', ylabel='Top 20 Kinases by Variance', yticklabels=True)
    clustergrammer_res = clustergrammer_link(dn_kea_comb.loc[top_20_var_kinases], f'{phospho_filename}_kea_dn_top20')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{phospho_filename}_kea_dn_top_var_heatmap.png', dpi=300)
    g.savefig(f'figures/{phospho_filename}_kea_dn_top_var_heatmap.svg', dpi=300)

In [ ]:
if has_phospho_file:    
    display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Integrated Mean Rank from KEA3 from down-regulated phosphosites for all samples and top 20 most variable kinases based on z-scored phosphorylation."))
    display(FileLink(f'figures/{phospho_filename}_kea_dn_top_var_heatmap.png', result_html_prefix='KEA3 Down Top 20 Variable Kinases Heatmap PNG: '))
    display(FileLink(f'figures/{phospho_filename}_kea_dn_top_var_heatmap.svg', result_html_prefix='KEA3 Down Top 20 Variable Kinases Heatmap SVG: '))

In [ ]:
if has_phospho_file:
    display(Markdown(f'KEA3 results were generated for the top {kea3_n} up and down-regulated phosphosites for each sample z-scored sample. The results were then aggregated and visualized as heatmaps for all kinases as well as the top 20 most variable kinases (Fig. {fig_counter}{ascii_uppercase[0]}-{ascii_uppercase[letter_counter]}).'))
    letter_counter = 0 
    fig_counter += 1

In [ ]:
%%appyter markdown
{% if rna_file.value != '' and phospho_file.value != '' %}
### Transcriptomics + Phosphoproteomics Analysis Results
#### Kinases enriched in both X2K and phospho- pipelines
{% endif %}

In [ ]:
if has_rna_file and has_phospho_file:
    rank_threshold = 30

    rec_kinases = {}
    rec_kinases_shuffled = {}
    shuffled_ids = list(kea_res.keys())
    np.random.shuffle(shuffled_ids)
    for i, id in enumerate(kea_res):
        if id not in x2k_up:
            continue
        up = x2k_up[id].astype(float).sort_values(ascending=True).index.to_list()
        up_phospho = kea_res[id]['up'].astype(float).sort_values(by='Score', ascending=True).index.to_list()
        up_phospho_rand = kea_res[shuffled_ids[i]]['up'].astype(float).sort_values(by='Score', ascending=True).index.to_list()
        rec_kinases[id] = list(set(up[:rank_threshold]) & set(up_phospho[:rank_threshold]))
        rec_kinases_shuffled[id] = list(set(up[:rank_threshold]) & set(up_phospho_rand[:rank_threshold]))
    up_up_df_shuffled = pd.DataFrame.from_dict(rec_kinases_shuffled, orient='index').T
    up_up_df = pd.DataFrame.from_dict(rec_kinases, orient='index').T
    up_up_df.to_csv(f"results/recovered_up_phospho_up_genes.csv", index=False)

    rec_kinases = {}
    rec_kinases_shuffled = {}
    shuffled_ids = list(kea_res.keys())
    np.random.shuffle(shuffled_ids)
    for i, id  in enumerate(kea_res):
        if id not in x2k_up:
            continue
        dn = x2k_dn[id].astype(float).sort_values(ascending=True).index.to_list()
        dn_phospho = kea_res[id]['down'].astype(float).sort_values(by='Score', ascending=True).index.to_list()
        dn_phospho_rand = kea_res[shuffled_ids[i]]['down'].astype(float).sort_values(by='Score', ascending=True).index.to_list()
        rec_kinases[id] = list(set(dn[:rank_threshold]) & set(dn_phospho[:rank_threshold]))
        rec_kinases_shuffled[id] = list(set(dn[:rank_threshold]) & set(dn_phospho_rand[:rank_threshold]))
    dn_dn_df_shuffled = pd.DataFrame.from_dict(rec_kinases_shuffled, orient='index').T
    dn_dn_df = pd.DataFrame.from_dict(rec_kinases, orient='index').T
    dn_dn_df.to_csv(f"results/recovered_dn_phospho_dn_genes.csv", index=False)

    rec_kinases = {}
    rec_kinases_shuffled = {}
    shuffled_ids = list(kea_res.keys())
    np.random.shuffle(shuffled_ids)
    for i, id in enumerate(kea_res):
        if id not in x2k_up:
            continue
        dn = x2k_dn[id].astype(float).sort_values(ascending=True).index.to_list()
        up_phospho = kea_res[id]['up'].astype(float).sort_values(by='Score', ascending=True).index.to_list()
        up_phospho_rand = kea_res[shuffled_ids[i]]['up'].astype(float).sort_values(by='Score', ascending=True).index.to_list()
        rec_kinases[id] = list(set(dn[:rank_threshold]) & set(up_phospho[:rank_threshold]))
        rec_kinases_shuffled[id] = list(set(dn[:rank_threshold]) & set(up_phospho_rand[:rank_threshold]))
    up_dn_df_shuffled = pd.DataFrame.from_dict(rec_kinases_shuffled, orient='index').T
    up_dn_df = pd.DataFrame.from_dict(rec_kinases, orient='index').T
    up_dn_df.to_csv(f"results/recovered_up_phospho_dn_genes.csv", index=False)

    rec_kinases = {}
    rec_kinases_shuffled = {}
    shuffled_ids = list(kea_res.keys())
    np.random.shuffle(shuffled_ids)
    for i, id in enumerate(kea_res):
        if id not in x2k_up:
            continue
        up = x2k_up[id].astype(float).sort_values(ascending=True).index.to_list()
        dn_phospho = kea_res[id]['down'].astype(float).sort_values(by='Score', ascending=True).index.to_list()
        dn_phospho_rand = kea_res[shuffled_ids[i]]['down'].astype(float).sort_values(by='Score', ascending=True).index.to_list()
        rec_kinases[id] = list(set(up[:rank_threshold]) & set(dn_phospho[:rank_threshold]))
        rec_kinases_shuffled[id] = list(set(up[:rank_threshold]) & set(dn_phospho_rand[:rank_threshold]))
    dn_up_df_shuffled = pd.DataFrame.from_dict(rec_kinases_shuffled, orient='index').T
    dn_up_df = pd.DataFrame.from_dict(rec_kinases, orient='index').T
    dn_up_df.to_csv(f"results/recovered_dn_phospho_up_genes.csv", index=False)

In [ ]:
if has_rna_file and has_phospho_file:   
    display(up_up_df.head())
    display(Markdown(f"__Table {table_counter}__ Kinases recovered from both the X2K pipeline and the phosphoproteomic data for up-genes and up-phosphosites."))
    table_counter += 1
    display(FileLink(f"results/recovered_up_phospho_up_genes.csv", result_html_prefix='Download Recovered Kinases (up-up):'))

In [ ]:
if has_rna_file and has_phospho_file:      
    display(dn_dn_df.head())
    display(Markdown(f"__Table {table_counter}__ Kinases recovered from both the X2K pipeline and the phosphoproteomic data for down-genes and down-phosphosites."))
    table_counter += 1
    display(FileLink(f"results/recovered_dn_phospho_dn_genes.csv", result_html_prefix='Download Recovered Kinases (down-down):'))

In [ ]:
if has_rna_file and has_phospho_file:      
    display(up_dn_df.head())
    display(Markdown(f"__Table {table_counter}__ Kinases recovered from both the X2K pipeline and the phosphoproteomic data for up-genes and down-phosphosites."))
    table_counter += 1
    display(FileLink(f"results/recovered_up_phospho_dn_genes.csv", result_html_prefix='Download Recovered Kinases (up-down):'))

In [ ]:
if has_rna_file and has_phospho_file:
    display(dn_up_df.head())
    display(Markdown(f"__Table {table_counter}__ Kinases recovered from both the X2K pipeline and the phosphoproteomic data for down-genes and up-phosphosites."))
    table_counter += 1
    display(FileLink(f"results/recovered_dn_phospho_up_genes.csv", result_html_prefix='Download Recovered Kinases (down-up):'))

In [ ]:
if has_rna_file and has_phospho_file:
    up_up = pd.read_csv("results/recovered_up_phospho_up_genes.csv")
    up_dn = pd.read_csv("results/recovered_up_phospho_dn_genes.csv")
    dn_up = pd.read_csv("results/recovered_dn_phospho_dn_genes.csv")
    dn_dn = pd.read_csv("results/recovered_dn_phospho_dn_genes.csv")

    samples_to_plot = []
    for cluster in sorted(leiden_df['leiden'].unique()):
        cluster_samples = list(set(leiden_df[leiden_df['leiden'] == cluster].index.values).intersection(up_up.columns))
        dat = []
        dat_shuffled = []
        for id in cluster_samples:
            dat.append([id, 'Up', 'Up', up_up.shape[0]-sum(up_up[id].isna())])
            dat.append([id, 'Up', 'Down', up_dn.shape[0]-sum(up_dn[id].isna())])
            dat.append([id, 'Down', 'Down', dn_dn.shape[0]-sum(dn_dn[id].isna())])
            dat.append([id, 'Down', 'Up', dn_up.shape[0]-sum(dn_up[id].isna())])
            dat_shuffled.append([id, 'Up', 'Up', up_up_df_shuffled.shape[0]-sum(up_up_df_shuffled[id].isna())])
            dat_shuffled.append([id, 'Up', 'Down', up_dn_df_shuffled.shape[0]-sum(up_dn_df_shuffled[id].isna())])
            dat_shuffled.append([id, 'Down', 'Down', dn_dn_df_shuffled.shape[0]-sum(dn_dn_df_shuffled[id].isna())])
            dat_shuffled.append([id, 'Down', 'Up', dn_up_df_shuffled.shape[0]-sum(dn_up_df_shuffled[id].isna())])

        dat = pd.DataFrame(dat, columns=["id", "Kinases", "Genes", "Overlap"])
        dat = dat.pivot(index=['id'], columns=['Kinases', 'Genes'], values='Overlap')
        dat_shuffled = pd.DataFrame(dat_shuffled, columns=["id", "Kinases", "Genes", "Overlap"])
        sorted_samples = dat.sum(axis=1).sort_values(ascending=False).index.values
        dat = dat.loc[sorted_samples]
        dat_shuffled = dat_shuffled.pivot(index=['id'], columns=['Kinases', 'Genes'], values='Overlap')
        dat.loc['random'] = dat_shuffled.loc[sorted_samples].mean()

        # Identify Samples with better than average recovery
        samp_p_values = [ttest_ind(dat.loc[samp], dat.loc['random'], alternative='greater') for samp in dat.drop('random').index.values]
        g = dat.plot(kind='bar', stacked=True, color=['coral', 'peachpuff', 'royalblue', 'lightblue'], fontsize=8, figsize=(len(up_up.columns)*0.15, 6))
        bars = g.patches
        top_bars_idx = (len(bars) // 4) -1
        for idx, bar in enumerate(bars):
            if ((idx + 1) % (len(bars) // 4)) == 0:
                bar.set_hatch('/')
        for i, bar in enumerate(g.containers[3][:-1]):
            if samp_p_values[i].pvalue < 0.0005:
                samples_to_plot.append((dat.index.values[i], samp_p_values[i].pvalue))
                plt.text(bar.xy[0] +(bar.get_width() / 4), bar.xy[1] + bar.get_height(), "***", fontsize=16)
            elif samp_p_values[i].pvalue < 0.005:
                samples_to_plot.append((dat.index.values[i], samp_p_values[i].pvalue))
                plt.text(bar.xy[0] +(bar.get_width() / 4), bar.xy[1] + bar.get_height(), "**", fontsize=16)
            elif samp_p_values[i].pvalue < 0.05:
                samples_to_plot.append((dat.index.values[i], samp_p_values[i].pvalue))
                plt.text(bar.xy[0] +(bar.get_width() / 4), bar.xy[1] + bar.get_height(), "*", fontsize=16)

            
        g.set_ylabel("Kinase Overlap")
        g.set_xlabel("Samples")
        #g.legend(title="Kinases,Genes", loc='best')
        plt.tight_layout()
        plt.savefig(f"figures/kinase_recovery_c{cluster}.svg", dpi=300)
        plt.savefig(f"figures/kinase_recovery_c{cluster}.png", dpi=300)
        plt.show()
        display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Overlap of kinases recovered from both the X2K pipeline and the phosphoproteomic data for up/down genes and phosphosites per patient in cluster {cluster}."))
        letter_counter += 1
        display(FileLink(f"figures/kinase_recovery_c{cluster}.png", result_html_prefix=f'Barchart of Kinase Recovery Cluster {cluster} PNG: '))

        display(FileLink(f"figures/kinase_recovery_c{cluster}.svg", result_html_prefix=f'Barchart of Kinase Recovery Cluster {cluster} SVG: '))

In [ ]:
if has_rna_file and has_phospho_file:
    display(Markdown(f"The recovery of kinases from both the X2K pipeline and the phosphoproteomic data for both up and down genes and phosphosites per patient in each cluster is compared to random recovery and visualized as a stacked bar chart, which each color matching a pair of directions of gene and phosphosite regulation (Fig. {fig_counter}{ascii_uppercase[0]}-{ascii_uppercase[letter_counter - 1]})."))
    letter_counter = 0
    fig_counter += 1

In [ ]:
%%appyter code_exec
{% if prot_file.value != '' and impute_protein_expr.raw_value %}
if has_prot_file:
    percentages = []
    idx = []
    i = 0
    for _, row in tqdm(prot_df.iterrows(), total=len(prot_df)):
        value_counts = row.astype(float).isna().value_counts()
        pr = value_counts[0]/len(row)
        if pr > percent_rows_protein:
            idx.append(i)
        percentages.append(pr)
        i += 1
    ax = pd.Series(percentages).hist()
    ax.set_ylabel("# Proteins")
    ax.set_xlabel("% with values")
{% endif %}

In [ ]:
%%appyter code_exec
{% if prot_file.value != '' and impute_protein_expr.raw_value %}
if has_prot_file:
    display(Markdown(f"__Fig. {fig_counter}__ Distribution of missing values in proteomic data."))
{% endif %}

In [ ]:
%%appyter code_exec
if has_prot_file:
    {% if impute_protein_expr.raw_value %}
    n1 = len(prot_df)
    prot_df_imputed = prot_df.iloc[idx].T.fillna(prot_df.iloc[idx].mean(axis=1)).T.astype(float)
    zscored_prot = zscore(prot_df_imputed, axis=1)
    zscored_samps = zscore(prot_df_imputed, axis=0)
    display(Markdown(f"Dropped {n1 - len(prot_df_imputed)} rows with missing values while imputing with averages. Distribution of missing values in proteomics data is displayed in Figure {fig_counter}."))
    fig_counter += 1
    {% else %}
    prot_df.dropna(inplace=True)
    zscored_prot = zscore(prot_df, axis=1)
    zscored_samps = zscore(prot_df, axis=0)
    {% endif %}

In [ ]:
if has_prot_file:
    top_var_proteins = zscored_samps.var(axis=1).sort_values(ascending=False)
    top_var_proteins_list = top_var_proteins.index[:100].tolist()
    df = pd.DataFrame(top_var_proteins, columns=['Variance'])
    df.index.names = ['Protein']
    display(df.head(10))
    display(Markdown(f"__Table {table_counter}__ Top 10 most variable proteins based on z-scored expression."))
    table_counter += 1

In [ ]:
%%appyter markdown
{% if prot_file.value != '' %}
### Top 100 Proteins by Variance
{% endif %}

In [ ]:
if has_prot_file:
    try:
        g = create_annotated_clustermap(zscored_prot.loc[top_var_proteins_list], meta_df, leiden_df, default_meta_cols, cbar_label='Z-Score', xlabel='Samples', ylabel='Top 100 Proteins by Variance')
    except:
        plt.clf()
        g = create_clustermap(zscored_prot.loc[top_var_proteins_list], cbar_label='Z-Score', xlabel='Samples', ylabel='Top 100 Proteins by Variance')
    clustergrammer_res = clustergrammer_link(zscored_prot.loc[top_var_proteins_list], f'{prot_filename}_zscored_top_var_proteins')
    if clustergrammer_res:
        display(Markdown(f"[View in Clustergrammer]({clustergrammer_res})"))
    g.savefig(f'figures/{prot_filename}_top_var_proteins_heatmap.png', dpi=300)
    g.savefig(f'figures/{prot_filename}_top_var_proteins_heatmap.svg', dpi=300)

In [ ]:
if has_prot_file:
    display(Markdown(f"__Fig. {fig_counter}__ Hierarchically clustered heatmap of the top 100 most variable proteins."))
    display(FileLink(f'figures/{prot_filename}_top_var_proteins_heatmap.png', result_html_prefix='Most Variable Proteins Heatmap PNG: '))
    display(FileLink(f'figures/{prot_filename}_top_var_proteins_heatmap.svg', result_html_prefix='Most Variable Proteins Heatmap SVG: '))

In [ ]:
if has_prot_file:
    display(Markdown(f"Top 100 most variable proteins were identified based on Z-scored expression and visualized (Fig. {fig_counter})."))
    fig_counter += 1

In [ ]:
if has_rna_file and has_prot_file:  
    adata = sc.AnnData(zscored_samps.T.values, dtype="float")
    adata.var['protein_names'] = zscored_samps.index.values
    adata.obs['samples'] = zscored_samps.columns.values
    adata.var['var_rank'] = (-np.var(adata.X, axis=0, dtype="float")).argsort()
    adata = adata[:, adata.var.var_rank < 5000]

    adata.obs['leiden'] = adata.obs['samples'].map(lambda s: leiden_df.loc[s]['leiden'] if s in leiden_df.index.values else 'N/A' )
    sc.pp.pca(adata)
    sc.pp.neighbors(adata, n_neighbors=n_neighbors)  # create neighborhood graph
    sc.tl.umap(adata, min_dist=min_dist, alpha=0.1) 
    clusters = adata.obs['leiden'].unique()
    cmap = plt.cm.get_cmap('tab20b', len(clusters))
    for i, cluster in enumerate(sorted(clusters)):
        mask = adata.obs['leiden'] == cluster
        ax = plt.scatter(x=adata.obsm['X_umap'][mask, 0], y=adata.obsm['X_umap'][mask, 1], 
                    color=cmap(i), label=f'Cluster {cluster}', s=10)
    plt.legend(title='Leiden clusters', bbox_to_anchor=(1.05, 1), loc='best')
    ax.axes.set_xticks([])
    ax.axes.set_yticks([])
    plt.xlabel('UMAP-1')
    plt.ylabel('UMAP-2')
    plt.savefig(f'figures/{prot_filename}_umap_leiden_rna_clusters.png', dpi=300, bbox_inches='tight')
    plt.savefig(f'figures/{prot_filename}_umap_leiden_rna_clusters.svg', dpi=300, bbox_inches='tight')
elif has_prot_file:
    adata = sc.AnnData(zscored_samps.T.values, dtype="int")
    adata.var['protein_names'] = zscored_samps.index.values
    adata.obs['samples'] = zscored_samps.columns.values
    # Sort proteins by variance
    adata.var['var_rank'] = (-np.var(adata.X, axis=0, dtype="float")).argsort()
    adata = adata[:, adata.var.var_rank < 5000]

    # UMAP
    sc.pp.pca(adata)
    sc.pp.neighbors(adata, n_neighbors=n_neighbors)  # create neighborhood graph
    sc.tl.umap(adata, min_dist=min_dist, alpha=0.1)  # embed umap based on neighborhood graph
    sc.tl.leiden(adata, resolution=resolution)  # clustering
    clusters = adata.obs['leiden'].unique()
    cmap = plt.cm.get_cmap('tab20b', len(clusters))
    for i, cluster in enumerate(sorted(clusters)):
        mask = adata.obs['leiden'] == cluster
        ax = plt.scatter(x=adata.obsm['X_umap'][mask, 0], y=adata.obsm['X_umap'][mask, 1], 
                    color=cmap(i), label=f'Cluster {cluster}', s=10)
    plt.legend(title='Leiden clusters', bbox_to_anchor=(1.05, 1), loc='best')
    ax.axes.set_xticks([])
    ax.axes.set_yticks([])
    plt.xlabel('UMAP-1')
    plt.ylabel('UMAP-2')
    plt.savefig(f'figures/{prot_filename}_umap_leiden.png', dpi=300, bbox_inches='tight')
    plt.savefig(f'figures/{prot_filename}_umap_leiden.svg', dpi=300, bbox_inches='tight')
    display(Markdown(f"__Fig. {fig_counter}__ Leiden clusters determined from Z-scored proteomic expression for 5000 most variable proteins."))
    display(FileLink(f'figures/{prot_filename}_umap_leiden.png', result_html_prefix='Leiden clusters UMAP PNG: '))
    display(FileLink(f'figures/{prot_filename}_umap_leiden.svg', result_html_prefix='Leiden clusters UMAP SVG: '))
    fig_counter += 1

In [ ]:
if has_rna_file and has_prot_file:
    display(Markdown(f"__Fig. {fig_counter}__ UMAP visualization of top 5000 most variable proteins colored by leiden clusters determined from RNA-seq expression."))
    display(FileLink(f'figures/{prot_filename}_umap_leiden_rna_clusters.png', result_html_prefix='Protein UMAP colored by RNA Leiden clusters PNG: '))
    display(FileLink(f'figures/{prot_filename}_umap_leiden_rna_clusters.svg', result_html_prefix='Protein UMAP colored by RNA Leiden clusters SVG: '))
elif has_prot_file:
    display(Markdown(f"__Fig. {fig_counter}__ Leiden clusters determined from Z-scored proteomic expression for 5000 most variable proteins."))
    display(FileLink(f'figures/{prot_filename}_umap_leiden.png', result_html_prefix='Leiden clusters UMAP PNG: '))
    display(FileLink(f'figures/{prot_filename}_umap_leiden.svg', result_html_prefix='Leiden clusters UMAP SVG: '))

In [ ]:
if has_rna_file and has_prot_file:
    display(Markdown(f"Proteomics expression was visualized using UMAP and colored by RNA-seq determined leiden clusters (Fig. {fig_counter})."))
    fig_counter += 1
elif has_prot_file:
    display(Markdown(f"Proteomics expression was visualized using UMAP and leiden clusters were determined for the top 5000 most variable proteins (Fig. {fig_counter})."))

In [ ]:
%%appyter markdown
{% if prot_file.value != '' and rna_file.value != '' %}
### Annotated Cell-Surface Targets with Proteomic Expression
{% endif %}

In [ ]:
if has_rna_file and has_prot_file:
    cmap = {"None":YlGnBu(0), "ARCHS4": YlGnBu(1), "GTEx":YlGnBu(2), "TS": YlGnBu(4), "ARCHS4-GTEx":YlGnBu(3),  "ARCHS4-TS": YlGnBu(5), "GTEx-TS": YlGnBu(6), "All": YlGnBu(7)}
    annot_data = []
    for t in membrane_target_mat.index:
        if t in zscored_prot.index:
            row = []
            for cluster in membrane_target_mat.columns:
                c = cluster.split(' ')[1]
                cluster_samples = list(leiden_df[leiden_df['leiden'] == c].index.values)
                cluster_samples = list(set(cluster_samples).intersection(zscored_prot.columns))
                clus_mean = (zscored_samps.loc[t][cluster_samples]).mean()
                row.append(str(clus_mean)[0:4])
            annot_data.append(row)
        else:
            annot_data.append(['NA']*membrane_target_mat.shape[1])
    h = membrane_target_mat.shape[0]
    g = sns.clustermap(membrane_target_mat, annot=annot_data, fmt='', figsize=(4,(0.3*h+2*(h<15))), cmap=YlGnBu, cbar_pos=None, dendrogram_ratio=0.1-(h<40)*0.01*(h-30), row_cluster=False, xticklabels=True, yticklabels=True, annot_kws={"fontsize":8})
    g.ax_row_dendrogram.legend(handles=[Rectangle((0, 0), 0, 0, color=val, label=key) for key, val in cmap.items()],
                                    title='Background', loc='upper right')

    plt.show()

    g.savefig(f'figures/{rna_filename}_prot_membrane_target_matrix.png', dpi=300)
    g.savefig(f'figures/{rna_filename}_prot_membrane_target_matrix.svg', dpi=300)
    membrane_target_mat.to_csv(f'results/{rna_filename}_prot_membrane_target_mat.csv')
    display(Markdown(f"__Fig. {fig_counter}__ Membrane target matrix annotated with average z-scored expression from proteomic data."))
    display(FileLink(f'results/{rna_filename}_prot_membrane_target_mat.csv', result_html_prefix='Download Membrane Target Matrix: '))
    display(FileLink(f'figures/{rna_filename}_prot_membrane_target_matrix.png', result_html_prefix='Membrane Target Matrix PNG: '))
    display(FileLink(f'figures/{rna_filename}_prot_membrane_target_matrix.svg', result_html_prefix='Membrane Target Matrix SVG: '))

In [ ]:
if has_rna_file and has_prot_file:
    display(Markdown(f"Membrane target matrix was annotated with average Z-scored expression across samples comprising each cluster using the uploaded proteomics data (Fig. {fig_counter})."))
    fig_counter += 1

In [ ]:
%%appyter markdown
{% if prot_file.value != '' and rna_file.value != '' and phospho_file.value != '' %}
### Recovered Kinases Proteomic Expression per Patient
{% endif %}

In [ ]:
if has_rna_file and has_prot_file and has_phospho_file:
    bar_width = .30
    sig_kinase_proteins = []
    for s, pval in samples_to_plot:
        up_up_dict = up_up_df.to_dict(orient='list')
        dn_dn_dict = dn_dn_df.to_dict(orient='list')
        up_dn_dict = up_dn_df.to_dict(orient='list')
        dn_up_dict = dn_up_df.to_dict(orient='list')
        rec_kinases_sample = list(set(up_up_dict[s] + dn_dn_dict[s] + up_dn_dict[s] + dn_up_dict[s]))
        idx_overlap = list(set(rec_kinases_sample).intersection(zscored_samps.index))
        prot_vals = zscored_samps[s].loc[idx_overlap].sort_values(ascending=False)
        prot_vals_avg = zscored_samps.loc[prot_vals.index].mean(axis=1)
        y_err_std = zscored_samps.loc[prot_vals.index].std(axis=1).values
        r1 = np.arange(len(prot_vals))
        r2 = [x + bar_width for x in r1]
        ax1 = plt.figure(figsize=(10, 6))
        ax2 = plt.bar(r1, prot_vals, width=bar_width, color='lightblue', label='Sample Protein Expression', ecolor='#a8a8a8', capsize=3)
        plt.bar(r2, prot_vals_avg, width=bar_width, color='grey', label='Mean Protein Expression', yerr=y_err_std, ecolor='#a8a8a8', capsize=3)
        for i, b in enumerate(ax2.patches):
            if prot_vals.iloc[i] > prot_vals_avg.iloc[i] +  abs((2 * y_err_std[i])):
                sig_kinase_proteins.append((prot_vals.index[i], 'up' if prot_vals.iloc[i] > prot_vals_avg.iloc[i] else 'down', s))
                plt.text(b.get_x() + b.get_width() / 2, b.get_height(), '*', ha='center', va='center', fontsize=16)
            elif prot_vals.iloc[i] < prot_vals_avg.iloc[i] - abs((2 * y_err_std[i])):
                sig_kinase_proteins.append((prot_vals.index[i], 'up' if prot_vals.iloc[i] > prot_vals_avg.iloc[i] else 'down', s))
                plt.text(b.get_x() + b.get_width() / 2, b.get_height(), '*', ha='center', va='center', fontsize=16)
        plt.xticks([r + bar_width for r in range(len(prot_vals.index))], prot_vals.index, rotation=90)
        plt.legend()
        plt.xlabel(f'Kinases Recovered for {s}')
        plt.ylabel('Z-Scored Protein Expression')
        plt.savefig(f'figures/{prot_filename}_prot_kinase_expr_{s}.png', dpi=300, bbox_inches='tight')
        plt.savefig(f'figures/{prot_filename}_prot_kinase_expr_{s}.svg', dpi=300, bbox_inches='tight')
        plt.show()
        display(Markdown(f"__Fig. {fig_counter}{ascii_uppercase[letter_counter]}__ Z-scored protein expression of overlapping kinases from X2K from up genes and directly from phosphoproteomics through KEA3 from up phosphosites."))
        letter_counter += 1 
        display(FileLink(f'figures/{prot_filename}_prot_kinase_expr_{s}.png', result_html_prefix=f'Z-scored protein expression of overlapping kinases from X2K and KEA3 for {s} PNG: '))
        display(FileLink(f'figures/{prot_filename}_prot_kinase_expr_{s}.svg', result_html_prefix=f'Z-scored protein expression of overlapping kinases from X2K and KEA3 for {s} SVG: '))

In [ ]:
if has_rna_file and has_prot_file and has_phospho_file:
    display(Markdown(f"Z-scored protein expression for kinases identified for patients that showed significantly better than random recovery of kinases in X2K when compared to the kinase enrichment from the phosphoproteomics data (Fig. {fig_counter}{ascii_uppercase[0]}-{ascii_uppercase[letter_counter]})."))
    letter_counter = 0
    fig_counter += 1

In [ ]:
if has_rna_file and has_prot_file and has_phospho_file:
    display(Markdown(create_results_text_prompt(sig_kinase_proteins, 'The following kinases were found to have significantly different protein expression compared to the average expression of the proteins across samples. This result was computed for samples that had greater than average recovery of kinases from X2K (RNA-seq expression to kinases) pipeline compared to kinases directly inferred from phosphoproteomics.')))

In [ ]:
if has_rna_file and has_prot_file and has_phospho_file:
    display(Markdown(f"*\*This section was generated with GPT-4 and should be interpreted with caution*\*"))

## Discussion

While the X2K pipeline predicted many kinases and TFs with well-studied individual roles in cancer, the signaling pathways and regulatory networks connecting are less well known. By inferring co-expressed intermediates and identifying kinases and TFs enriched in the same tumor sample, the X2K analysis presents possible novel regulatory pathways linking upstream kinases to their downstream targets.

The target identification component of the pipeline also facilitates hypothesis generation regarding novel cancer targets. Many predicted targets are not directly linked to the specific cancer subtype, but rather connected to other cancers, immune processes, or mechanisms related to cell proliferation and cancer cell survival.

## Conclusions

Finding targets and key to cell signaling pathways that are uniquely activated in tumor cells while absent from all normal tissues and cell types remains a significant challenge. In addition, targets identified from analyzing transcriptomics do not ensure that the targets are also expressed at the protein level. Furthermore, heterogeneity within the tumor, across patients of the same cancer type, and across cancers, underscore the need for subtype-specific personalized target identification. Our analysis and workflows address some of these challenges by combining phosphoproteomics, proteomics, and transcriptomics with comprehensive normal tissue and cell type atlases to identify safer targets for cancer subtypes.

## References

[1] Chen EY, Xu H, Gordonov S, Lim MP, Perkins MH, Ma'ayan A. Expression2Kinases: mRNA profiling linked to multiple upstream regulatory layers. Bioinformatics. 2012 Jan 1;28(1):105-11. doi: 10.1093/bioinformatics/btr625. Epub 2011 Nov 10.

[2] Clarke DJB, Kuleshov MV, Schilder BM, Torre D, Duffy ME, Keenan AB, Lachmann A, Feldmann AS, Gundersen GW, Silverstein MC, Wang Z, Ma'ayan A. eXpression2Kinases (X2K) Web: linking expression signatures to upstream cell signaling networks. Nucleic Acids Res. 2018 Jul 2;46(W1):W171-W179. doi: 10.1093/nar/gky458.

[2] Marino GB, Ngai M, Clarke DJB, Fleishman RH, Deng EZ, Xie Z, Ahmed N, Ma'ayan A. GeneRanger and TargetRanger: processed gene and protein expression levels across cells and tissues for target discovery. Nucleic Acids Res. 2023 Jul 5;51(W1):W213-W224. doi: 10.1093/nar/gkad399.

[4] Cancer Genome Atlas Research Network; Van Allen EM, Cherniack AD, Ciriello G, Sander C, Schultz N. Oncogenic Signaling Pathways in The Cancer Genome Atlas. Cell. 2018 Apr 5;173(2):321-337.e10. doi: 10.1016/j.cell.2018.03.035. PMID: 29625050; PMCID: PMC6070353.

[5] Keenan AB, Torre D, Lachmann A, Leong AK, Wojciechowicz ML, Utti V, Jagodnik KM, Kropiwnicki E, Wang Z, Ma'ayan A. ChEA3: transcription factor enrichment analysis by orthogonal omics integration. Nucleic Acids Res. 2019 Jul 2;47(W1):W212-W224. doi: 10.1093/nar/gkz446.

[6] Berger SI, Posner JM, Ma'ayan A. Genes2Networks: connecting lists of gene symbols using mammalian protein interactions databases. BMC Bioinformatics. 2007 Oct 4;8:372. doi: 10.1186/1471-2105-8-372.

[7] Kuleshov MV, Xie Z, London ABK, Yang J, Evangelista JE, Lachmann A, Shu I, Torre D, Ma'ayan A. KEA3: improved kinase enrichment analysis via data integration. Nucleic Acids Res. 2021 Jul 2;49(W1):W304-W316. doi: 10.1093/nar/gkab359.

[8] Ge SX, Son EW, Yao R. iDEP: an integrated web application for differential expression and pathway analysis of RNA-Seq data. BMC Bioinformatics. 2018 Dec 19;19(1):534. doi: 10.1186/s12859-018-2486-6.

[9] Reimand J, Isserlin R, Voisin V, Kucera M, Tannus-Lopes C, Rostamianfar A, Wadi L, Meyer M, Wong J, Xu C, Merico D, Bader GD. Pathway enrichment analysis and visualization of omics data using g:Profiler, GSEA, Cytoscape and EnrichmentMap. Nat Protoc. 2019 Feb;14(2):482-517. doi: 10.1038/s41596-018-0103-9.

[10] Jin Y, Ratnam K, Chuang PY, Fan Y, Zhong Y, Dai Y, Mazloom AR, Chen EY, D'Agati V, Xiong H, Ross MJ, Chen N, Ma'ayan A, He JC. A systems approach identifies HIPK2 as a key regulator of kidney fibrosis. Nat Med. 2012 Mar 11;18(4):580-8. doi: 10.1038/nm.2685. Erratum in: Nat Med. 2021 Aug;27(8):1483.

[11] Bosse KR, Raman P, Zhu Z, Lane M, Martinez D, Heitzeneder S, Rathi KS, Kendsersky NM, Randall M, Donovan L, Morrissy S, Sussman RT, Zhelev DV, Feng Y, Wang Y, Hwang J, Lopez G, Harenza JL, Wei JS, Pawel B, Bhatti T, Santi M, Ganguly A, Khan J, Marra MA, Taylor MD, Dimitrov DS, Mackall CL, Maris JM. Identification of GPC2 as an Oncoprotein and Candidate Immunotherapeutic Target in High-Risk Neuroblastoma. Cancer Cell. 2017 Sep 11;32(3):295-309.e12. doi: 10.1016/j.ccell.2017.08.003.

[12] GTEx Consortium. The Genotype-Tissue Expression (GTEx) project. Nat Genet. 2013 Jun;45(6):580-5. doi: 10.1038/ng.2653.

[13] Peters C, Brown S. Antibody-drug conjugates as novel anti-cancer chemotherapeutics. Biosci Rep. 2015 Jun 12;35(4):e00225. doi: 10.1042/BSR20150089.

[14] Sadelain M, Rivière I, Brentjens R. Targeting tumours with genetically enhanced T lymphocytes. Nat Rev Cancer. 2003 Jan;3(1):35-45. doi: 10.1038/nrc971.

[15] Ho WY, Blattman JN, Dossett ML, Yee C, Greenberg PD. Adoptive immunotherapy: engineering T cell responses as biologic weapons for tumor mass destruction. Cancer Cell. 2003 May;3(5):431-7. doi: 10.1016/s1535-6108(03)00113-2.

[16] Orentas RJ, Yang JJ, Wen X, Wei JS, Mackall CL, Khan J. Identification of cell surface proteins as potential immunotherapy targets in 12 pediatric cancers. Front Oncol. 2012 Dec 17;2:194. doi: 10.3389/fonc.2012.00194.

[17] Lee JK, Bangayan NJ, Chai T, Smith BA, Pariva TE, Yun S, Vashisht A, Zhang Q, Park JW, Corey E, Huang J, Graeber TG, Wohlschlegel J, Witte ON. Systemic surfaceome profiling identifies target antigens for immune-based therapy in subtypes of advanced prostate cancer. Proc Natl Acad Sci U S A. 2018 May 8;115(19):E4473-E4482. doi: 10.1073/pnas.1802354115. Epub 2018 Apr 23.

[18] Ferguson ID, Patiño-Escobar B, Tuomivaara ST, Lin YT, Nix MA, Leung KK, Kasap C, Ramos E, Nieves Vasquez W, Talbot A, Hale M, Naik A, Kishishita A, Choudhry P, Lopez-Girona A, Miao W, Wong SW, Wolf JL, Martin TG 3rd, Shah N, Vandenberg S, Prakash S, Besse L, Driessen C, Posey AD Jr, Mullins RD, Eyquem J, Wells JA, Wiita AP. The surfaceome of multiple myeloma cells suggests potential immunotherapeutic strategies and protein markers of drug resistance. Nat Commun. 2022 Jul 15;13(1):4121. doi: 10.1038/s41467-022-31810-6.

[19] Lachmann A, Torre D, Keenan AB, Jagodnik KM, Lee HJ, Wang L, Silverstein MC, Ma'ayan A. Massive mining of publicly available RNA-seq data from human and mouse. Nature Communications 9. Article number: 1366 (2018), doi: 10.1038/s41467-018-03751-6.

[20] Tabula Sapiens Consortium. The Tabula Sapiens: A multiple-organ, single-cell transcriptomic atlas of humans. Science. 2022 May 13;376(6594):eabl4896. doi: 10.1126/science.abl4896

[21] Traag VA, Waltman L, van Eck NJ. From Louvain to Leiden: guaranteeing well-connected communities. Sci Rep. 2019 Mar 26;9(1):5233. doi: 10.1038/s41598-019-41695-z.

[22] Ritchie ME, Phipson B, Wu D, Hu Y, Law CW, Shi W, Smyth GK. limma powers differential expression analyses for RNA-sequencing and microarray studies. Nucleic Acids Res. 2015 Apr 20;43(7):e47. doi: 10.1093/nar/gkv007.

[23] Chen EY, Tan CM, Kou Y, Duan Q, Wang Z, Meirelles GV, Clark NR, Ma'ayan A. Enrichr: interactive and collaborative HTML5 gene list enrichment analysis tool. BMC Bioinformatics. 2013 Apr 15;14:128. doi: 10.1186/1471-2105-14-128.

[24] Ashburner M, Ball CA, Blake JA, Botstein D, Butler H, Cherry JM, Davis AP, Dolinski K, Dwight SS, Eppig JT, Harris MA, Hill DP, Issel-Tarver L, Kasarskis A, Lewis S, Matese JC, Richardson JE, Ringwald M, Rubin GM, Sherlock G. Gene ontology: tool for the unification of biology. The Gene Ontology Consortium. Nat Genet. 2000 May;25(1):25-9. doi: 10.1038/75556.

[25] Pico AR, Kelder T, van Iersel MP, Hanspers K, Conklin BR, Evelo C. WikiPathways: pathway editing for the people. PLoS Biol. 2008 Jul 22;6(7):e184. doi: 10.1371/journal.pbio.0060184.

[26] Eppig JT, Smith CL, Blake JA, Ringwald M, Kadin JA, Richardson JE, Bult CJ. Mouse Genome Informatics (MGI): Resources for Mining Mouse Genetic, Genomic, and Biological Data in Support of Primary and Translational Research. Methods Mol Biol. 2017;1488:47-73. doi: 10.1007/978-1-4939-6427-7_3.

[27]  Binder JX, Pletscher-Frankild S, Tsafou K, Stolte C, O'Donoghue SI, Schneider R, Jensen LJ. COMPARTMENTS: unification and visualization of protein subcellular localization evidence. Database (Oxford). 2014 Feb 25;2014:bau012. doi: 10.1093/database/bau012.

[28] Thul PJ, Lindskog C. The human protein atlas: A spatial map of the human proteome. Protein Sci. 2018 Jan;27(1):233-244. doi: 10.1002/pro.3307. Epub 2017 Oct 10.

[29] Nusinow DP, Szpyt J, Ghandi M, Rose CM, McDonald ER 3rd, Kalocsay M, Jané-Valbuena J, Gelfand E, Schweppe DK, Jedrychowski M, Golji J, Porter DA, Rejtar T, Wang YK, Kryukov GV, Stegmeier F, Erickson BK, Garraway LA, Sellers WR, Gygi SP. Quantitative Proteomics of the Cancer Cell Line Encyclopedia. Cell. 2020 Jan 23;180(2):387-402.e16. doi: 10.1016/j.cell.2019.12.023.

[30] Tsherniak A, Vazquez F, Montgomery PG, Weir BA, Kryukov G, Cowley GS, Gill S, Harrington WF, Pantel S, Krill-Burger JM, Meyers RM, Ali L, Goodale A, Lee Y, Jiang G, Hsiao J, Gerath WFJ, Howell S, Merkel E, Ghandi M, Garraway LA, Root DE, Golub TR, Boehm JS, Hahn WC. Defining a Cancer Dependency Map. Cell. 2017 Jul 27;170(3):564-576.e16. doi: 10.1016/j.cell.2017.06.010.